
# Requried libraries

In [2]:
import numpy as np
import pandas as pd
from math import sin, cos, sqrt, atan2, radians
import netCDF4 as nc
from netCDF4 import Dataset
import math
from scipy import ndimage
from skimage import measure
from statistics import mode, mean
import shutil
import os
from itertools import combinations
from scipy.interpolate import griddata
from os import path



# Driver for the Topographically InformEd Regression (TIER) tool

This tool is built on the concept that geophyiscal attributes contain information useful to predict the spatial distribution of meteorological variables (e.g. Daly et al. 1994,2002,2007,2008; Thornton et al. 1999; Clark and Slater 2006; Carrera-Hernandez and Gaskin 2007; Tobin et al. 2011; Bardossy and Pegram 2013; Newman et al. 2015).

A pre-processed DEM file (see tierPreprocessing.m) and input station data are used to create monthly climatological meteorological fields over a domain.  

Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
Email : anewman@ucar.edu & mozhgana@ucar.edu

Copyright (C) 2019 University Corporation for Atmospheric Research

This file is part of TIER.

TIER is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or(at your option) any later version.
TIER is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty o MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.
You should have received a copy of the GNU General Public License along with TIER.  If not, see <https://www.gnu.org/licenses/>.
 


# User determines the controlName

In [4]:
controlName = input('Enter the name of your control file: ')

Enter the name of your control file: tierControl.txt


# readControl:
  
* **Reads a text control file for TIER**

In [5]:
def readControl(controlName):
    
    """
    readControl reads a text control file for TIER
    TIER - Topographically InformEd Regression

     Arguments:

     Input:controlName, string, the name of the grid file

     Output:controlVars, structure
     , stucture holding all control variables
                           
        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu
 
     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.
    """
   #open control file
    fid = open(controlName,'r')
    #read data
    data = np.loadtxt(fid, delimiter=',',skiprows=1,dtype=str, usecols=[0,1,2])
    #close control file
    fid.close()

    class structtype():
        pass
    controlVars = structtype()

    #run through all lines in control file
    for i in range(0,len(data)):
        #test string name and place in appropriate named variable
        if [row[0] for row in data][i]== 'gridName':
            controlVars.gridName = [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='stationFileList':
            controlVars.stationFileList  = [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='stationDataPath':
            controlVars.stationDataPath = [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='outputName':
            controlVars.outputName = [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='parameterFile':
            controlVars.parameterFile= [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='variableEstimated':
            controlVars.variableEstimated = [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='defaultTempLapse':
            controlVars.defaultTempLapse = [row[1] for row in data][i].strip()

        else:
            #throw error if unknown string
            print('Unknown control file option: ' +[row[0] for row in data][i])

    return controlVars


In [6]:
#read control file
controlVars = readControl(controlName)
controlVars

<__main__.readControl.<locals>.structtype at 0x1b41565dd00>

# initPreprocessParameters:
* **Initalizes TIER parameters to defaults**

In [7]:
def initParameters(varEstimated):
    """

     initParameters initalizes TIER parameters to defaults
     TIER - Topographically InformEd Regression

     Arguments:

      Output: parameters, structure, structure holding all TIER parameters

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.
     
    """
    
    class structtype():
        pass
    parameters = structtype()
    #initialize all parameters to initial default value
    parameters.nMaxNear = 7                           #number
    parameters.nMinNear = 3                           #number
    parameters.maxDist = 250                          #km
    parameters.minSlope = 0.5                         #normalized for precipitation
    parameters.maxInitialSlope = 4.25                 #normalized
    parameters.maxFinalSlope = 3.0                    #normalized
    parameters.maxSlopeLower = 0                      #K/km
    parameters.maxSlopeUpper = 20                     #K/km
    parameters.defaultSlope = 1.3                     #normalized
    parameters.topoPosMinDiff = 500                   #m
    parameters.topoPosMaxDiff = 5000                  #m
    parameters.topoPosExp = 0.7                       # -
    parameters.coastalExp = 1.0                       # -
    parameters.layerExp = 0.5                         # -
    parameters.distanceWeightScale = 16000            # -
    parameters.distanceWeightExp = 2                  # -
    parameters.maxGrad = 2.5                          # normalized slope per grid cell
    parameters.bufferSlope = 0.02                     # normalized
    parameters.minElev = 100                          # m
    parameters.minElevDiff = 500                      # m
    parameters.filterSize = 60                        # grid cells
    parameters.filterSpread = 40                      # -
    parameters.covWindow = 10                         # grid cells
    parameters.recomputeDefaultPrecipSlope = 'false'  #logical
    parameters.recomputeDefaultTempSlope   = 'false'  #logical

    #initialize variables based on meteorological variable being regressed
    if varEstimated=='tmax' or varEstimated=='tmin':
        #temperature specific initialization values here
        parameters.nMaxNear = 30                 #number
        parameters.nMinNear = 3                  #number
        parameters.maxDist = 300                 #km
        parameters.minSlope = -10                #K/km
        parameters.maxSlopeLower = 0             #K/km
        parameters.maxSlopeUpper = 20            #K/km
        parameters.defaultSlope = -5             #normalized
        parameters.topoPosMinDiff = 0            #m 
        parameters.topoPosMaxDiff = 500          #m
        parameters.topoPosExp = 0.50             # -
        parameters.coastalExp = 1.0              # -
        parameters.layerExp = 4.0                # -
        parameters.distanceWeightScale = 20000   # - 
        
#     elif varEstimated =='precip':
#       #precipitation specific initialization values here 
    return parameters

In [8]:
#initalize parameters to default values
parameters = initParameters(controlVars.variableEstimated)
parameters.nMaxNear

7

# readParameters:
* **Reads a text parameter file for TIER**
* **Overrides the default values if parameters are present in parameter file**

In [9]:
def readParameters(parameterFile,parameters):
    """
    
    readParameters reads a text parameter file for TIER and overrides the default values 
    if parameters are present in parameter file

     TIER - Topographically InformEd Regression

     Arguments:

     Input: parameterFile, string, the name of the TIER parameter file
      parameters, structure, structure holding all TIER parameters

     Output:parameters, structure, structure holding all TIER parameters

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
 
  #open parameter file
    fid = open(parameterFile,'r')
    #read data
    data = np.loadtxt(fid, delimiter=',',skiprows=1,dtype=str, usecols=[0,1,2])
    #close control file
    fid.close()   
    
    #run through all lines in parameter file
    for i in range(0,len(data)):
        #test string name and place in appropriate named variable
        if [row[0] for row in data][i]== 'nMaxNear':
        #maximum number of nearby stations to consider            
            parameters.nMaxNear = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]== 'nMinNear':
            #minimum number of nearby stations needed for slope regression
            parameters.nMinNear = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='maxDist':
            #maximum distance to consider stations
            parameters.maxDist = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='minSlope':
            #minimum valid slope value (normalized for precipitation, physical units for temperature)
            parameters.minSlope = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='maxInitialSlope':
            #maximum valid initial pass normalized slope for precipitation
            parameters.maxInitialSlope = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='maxFinalSlope':
            #maximum valid final adjusted normalized slope for precipitation
            parameters.maxFinalSlope = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='maxSlopeLower':
            #maximum valid slope for temperature in lower atmospheric layer (inversion layer, allows for strong inversions)
            parameters.maxSlopeLower = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='maxSlopeUpper':
            #maximum valid slope for temperature in upper layer (free atmosphere, up to isothermal allowed)
            parameters.maxSlopeUpper = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='defaultSlope':
            #default slope value (normalized for precipitation, physical units for temperature)
            parameters.defaultSlope = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='topoPosMinDiff':
            #minimum elevation difference used to adjust topographic position weights
            parameters.topoPosMinDiff = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='topoPosMaxDiff':
            #maximum elevation difference for stations to receive topographic position weighting
            parameters.topoPosMaxDiff = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='topoPosExp':
            #exponent to adjust topographic position weighting function
            parameters.topoPosExp = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='coastalExp':
            #exponent to adjust distance to coast weighting function
            parameters.coastalExp = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='layerExp':
            #exponent to adjust atmospheric layer weighting function
            parameters.layerExp = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='distanceWeightScale':
            #scale parameter in Barnes (1964) distance weighting function
            parameters.distanceWeightScale = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='distanceWeightExp':
            #exponent in Barnes (1964) distance weighting function
            parameters.distanceWeightExp = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='maxGrad':
            #maximum allowable normalized precipitation slope gradient between grid cells
            parameters.maxGrad = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='bufferSlope':
            #a buffer parameter when computing precipitaiton slope feathering
            parameters.bufferSlope = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]== 'minElev':
            #minimum elevation considered when feathering precipitation
            parameters.minElev = float([row[1] for row in data][i].strip())/1000 #convert m to km
        elif [row[0] for row in data][i]=='minElevDiff':
            #minimum elevation difference across precipitation considered for feathering precipitation
            parameters.minElevDiff = float([row[1] for row in data][i].strip())/1000 #convert m to km
        elif [row[0] for row in data][i]=='recomputeDefaultPrecipSlope':
            #logical to indicate if the default slope should be
            #recomputed using the domain valid regression points            
            parameters.recomputeDefaultPrecipSlope = [row[1] for row in data][i].strip()
        elif [row[0] for row in data][i]=='recomputeDefaultTempSlope':
            #logical to indicate if the default slope should be
            #recomputed using the domain valid regression points
            parameters.recomputeDefaultTempSlope = [row[1] for row in data][i].strip()
        elif [row[0] for row in data][i]=='filterSize':
            #size of low pass filter (grid points) used in computing updated slopes and uncertainty estimates
            parameters.filterSize = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='filterSpread':
            #spread of low-pass filter power used in computing updated slopes and uncertainty estimates
            parameters.filterSpread = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='covWindow':
            #window for local covariance calculation for the baseInterp and slope uncertainty components.  Used in the final uncertainty estimation routine
            parameters.covWindow = float([row[1] for row in data][i].strip())
        
        else:
            #throw error if unknown string
            print('Unknown parameter name : '+[row[0] for row in data][i])
                  
    
    return parameters

In [10]:
#read user defined TIER parameter file
parameters = readParameters(controlVars.parameterFile,parameters)

In [12]:
parameters.nMinNear

3.0

# readGrid:
* **reads a netcdf grid file for the TIER code**

In [10]:
def readGrid(gridName):

    """
     readGrid reads a netcdf grid file for the TIER code
     TIER - Topographically InformEd Regression

     Arguments:

     Input:gridName, string, the name of the grid file

     Output:grid, structure, structure holding DEM, geophysical attributes
                       and related variables

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
        
    class structtype():
        pass
    grid = structtype()
    
    #loading dataset by passing a NetCDF file path
    ds = Dataset(gridName)
    
    #read latitude and longitude
    grid.lat = ds['latitude'][:]
    grid.lon = ds['longitude'][:]
    #grid spacing
    gridUnits = ds['dx'].units
    
    if gridUnits.lower()=='degrees':
        grid.dx = ds['dx'][:]
        #convert to km (roughly)
        
        #grid lower left lat,lon
        startLon = ds['startx'][:]
        startLat = ds['starty'][:]


        # phi = 90 - latitude
        phi1 = np.deg2rad(90.0 - startLat)
        phi2 = np.deg2rad(90.0 - (startLat+grid.dx))

        # theta = longitude
        theta1 = np.deg2rad(startLon)
        theta2 = np.deg2rad(startLon+grid.dx)


        # kmPerLat = math.sqrt((startLat - (startLat+grid.dx))**2 + (startLon- (startLon+grid.dx))**2)
        kmPerLat = sqrt((phi1 - phi2)**2 + (theta1- theta2)**2)*6371
        # distance = earth radius * radians
        
        # distance = earth radius * radians
        Earth_radius = 6371
        km2rad= kmPerLat/Earth_radius
        gridDist = np.rad2deg(km2rad)
        
        #grid distance in km roughly
        grid.dx = (grid.dx*kmPerLat)/sqrt(gridDist**2)
        
    elif gridUnits.lower() =='km':
        grid.dx = ds['dx'] 
    elif gridUnits.lower() =='m':
        grid.dx = ds['dx']/1000 #convert m to km
    else:
        print('Unknown grid dx units: ' + gridUnits+'\n')
    
    #read valid grid point mask
    grid.mask = ds['mask'][:] 
    #read DEM
    grid.dem = ds['elev'][:]
    #read smoothed DEM
    grid.smoothDem = ds['smooth_elev'][:]
    #read distance to coast
    grid.distToCoast = ds['dist_to_coast'][:]
    #read inversion layer
    grid.layerMask = ds['inversion_layer'][:]
    #read topographic position
    grid.topoPosition = ds['topo_position'][:]

    #convert DEM to km
    grid.dem = grid.dem/1000.0
    
    #read slope facet
    grid.facet = ds['facet'][:]
    #double check missing data points, reset very low DEM values to missing
    grid.facet[grid.dem < -100] = -999
    
    #set grid size variables
    (grid.nc,grid.nr) = np.shape(grid.lat)
    
    return grid
    

In [11]:
#read grid file
grid = readGrid(controlVars.gridName)
grid.smoothDemKM = grid.smoothDem/1000
grid.smoothDemKM

masked_array(
  data=[[--, --, --, ..., 0.9143148502111434, 0.859966135263443,
         0.6705146512389183],
        [--, --, --, ..., 0.8751585264205932, 0.8193460587859154,
         0.636209140777588],
        [--, --, --, ..., 0.8987839505672455, 0.8228409062623978,
         0.6274272040724754],
        ...,
        [--, --, --, ..., 1.2244391309022904, 1.106477852642536,
         0.8316656767725944],
        [--, --, --, ..., 1.1270257021784782, 1.0352777718901633,
         0.79061106133461],
        [--, --, --, ..., 1.0157711544036865, 0.9459361028075218,
         0.7316063257455826]],
  mask=[[ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        ...,
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False]],
  fill_value=-999.0)

# allocateMetVars:
* **allocates memory for met variables**

In [12]:
def allocateMetVars(nc,nr):
    """
    
    allocateMetVars allocates memory for met variables

     Arguments:

      Input:nr, integer, number of rows in grid
            nc, integer, number of columns in grid

      Output:metGrid, structure, structure housing all grids related to
                           met field generation

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.
     
     """
    class structtype():
        pass
    metGrid = structtype()
    #allocate space for grids
    
    metGrid.rawField         = np.zeros([nc,nr])* np.nan
    metGrid.intercept        = np.zeros([nc,nr])* np.nan
    metGrid.slope            = np.zeros([nc,nr])* np.nan
    metGrid.normSlope        = np.zeros([nc,nr])* np.nan
    metGrid.baseInterpField  = np.zeros([nc,nr])* np.nan
    metGrid.baseInterpElev   = np.zeros([nc,nr])* np.nan
    metGrid.baseInterpUncert = np.zeros([nc,nr])* np.nan
    metGrid.slopeUncert      = np.zeros([nc,nr])* np.nan
    metGrid.normSlopeUncert  = np.zeros([nc,nr])* np.nan
    metGrid.defaultSlope     = np.zeros([nc,nr])* np.nan
    metGrid.finalSlope       = np.zeros([nc,nr])* np.nan
    metGrid.finalField       = np.zeros([nc,nr])* np.nan
    metGrid.totalUncert      = np.zeros([nc,nr])* np.nan
    metGrid.relUncert        = np.zeros([nc,nr])* np.nan
    metGrid.validRegress     = np.zeros([nc,nr])* np.nan

    return metGrid 

In [13]:
#allocate space for output variables
metGrid = allocateMetVars(grid.nc,grid.nr)

In [14]:
np.shape(metGrid.normSlopeUncert)

(136, 128)

# readInputStations:
* **reads the input point station metadata**

In [15]:
 def readInputStations(controlVars):
    """"
    readInputStations reads the input point station metadata
    and station data for TIER
    TIER - Topographically InformEd Regression

     Arguments:

     Input:   stationFileList, string, the name of the station metadata file
              stationDataPath, string, path to location of station data
              controlVars, structure, structure holding control variables

     Output: inputStations, structure, structure holding input station data, metdata, etc

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    class structtype():
        pass
    meta= structtype()
    inputStations= structtype()
    #open station list file
    fid = open(controlVars.stationFileList,'r')
    #read data
    data = np.loadtxt(fid, delimiter=',',skiprows=2,dtype=str, usecols=[0,1,2,3,4,5,6,7,8])
    #close control file
    
    meta.staId = [row[0].strip() for row in data]
    meta.lat = [float(row[1].strip()) for row in data]
    meta.lon = [float(row[2].strip()) for row in data]
    meta.elev = [float(row[3].strip()) for row in data]
    meta.facet = [float(row[4].strip()) for row in data]
    meta.coastDist = [float(row[5].strip()) for row in data]
    meta.layer = [row[6].strip() for row in data]
    meta.topoPosition = [float(row[7].strip()) for row in data]
    meta.staName = [row[8].strip() for row in data]
    inputStations.meta= meta

    #close control file
    fid.close()

    #set number of stations
    nSta = len(inputStations.meta.staName)

    #convert elevation to km
    inputStations.meta.elev = [elev/1000 for elev in inputStations.meta.elev]

    #allocate inputStations structure
    inputStations.avgVar = np.zeros([nSta,1])

    if controlVars.variableEstimated=='precip':
        metVar = 'prcp'
    elif [controlVars.variableEstimated=='tmax'] or [controlVars.variableEstimated=='tmin']:
        metVar = np.lower(controlVars.variableEstimated)
    metVar

    #read data
    for i in range(0,nSta):
        print('Loading: %s\n'%inputStations.meta.staName[i])
        #create file name string    
        fname= '%s%s.nc'%(controlVars.stationDataPath,inputStations.meta.staName[i])
        #read station data
        inputStations.avgVar[i] = Dataset(fname)[metVar][:]

    return inputStations


In [16]:
#read input station data
inputStations = readInputStations(controlVars)

Loading: US1CAAL0001

Loading: US1CAAL0003

Loading: US1CAAL0004

Loading: US1CAAL0007

Loading: US1CAAL0011

Loading: US1CAAL0012

Loading: US1CAAM0003

Loading: US1CABT0002

Loading: US1CABT0007

Loading: US1CACC0001

Loading: US1CACC0004

Loading: US1CACC0010

Loading: US1CACC0011

Loading: US1CACC0016

Loading: US1CACC0018

Loading: US1CACV0002

Loading: US1CADN0001

Loading: US1CAED0010

Loading: US1CAFR0002

Loading: US1CAFR0003

Loading: US1CAFR0005

Loading: US1CAFR0007

Loading: US1CAFR0008

Loading: US1CAHM0001

Loading: US1CAHM0002

Loading: US1CAHM0003

Loading: US1CAHM0004

Loading: US1CAHM0005

Loading: US1CAHM0009

Loading: US1CAHM0014

Loading: US1CAHM0016

Loading: US1CAHM0021

Loading: US1CAHM0022

Loading: US1CAHM0026

Loading: US1CAHM0029

Loading: US1CAHM0030

Loading: US1CAHM0035

Loading: US1CAHM0038

Loading: US1CAIN0001

Loading: US1CAIN0002

Loading: US1CAKG0001

Loading: US1CAKG0003

Loading: US1CAKG0004

Loading: US1CAKG0005

Loading: US1CAKN0001

Loading: U

Loading: USC00044957

Loading: USC00044997

Loading: USC00045026

Loading: USC00045032

Loading: USC00045100

Loading: USC00045118

Loading: USC00045119

Loading: USC00045120

Loading: USC00045123

Loading: USC00045125

Loading: USC00045233

Loading: USC00045280

Loading: USC00045311

Loading: USC00045338

Loading: USC00045352

Loading: USC00045356

Loading: USC00045360

Loading: USC00045378

Loading: USC00045385

Loading: USC00045400

Loading: USC00045449

Loading: USC00045532

Loading: USC00045598

Loading: USC00045679

Loading: USC00045756

Loading: USC00045779

Loading: USC00045795

Loading: USC00045802

Loading: USC00045853

Loading: USC00045866

Loading: USC00045915

Loading: USC00045933

Loading: USC00045941

Loading: USC00046027

Loading: USC00046074

Loading: USC00046136

Loading: USC00046144

Loading: USC00046168

Loading: USC00046172

Loading: USC00046174

Loading: USC00046194

Loading: USC00046252

Loading: USC00046328

Loading: USC00046329

Loading: USC00046336

Loading: U

In [17]:
#if temperature and a gridded default slope file is specified, read it
if (controlVars.variableEstimated=='tmax') & (len(controlVars.defaultTempLapse)!=0):     
    tempDefaultLapse = Dataset(controlVars.defaultTempLapse)['tmaxLapse'][:]
    #override user specified choice for the recomputeDefaultTempSlope option
    parameters.recomputeDefaultTempSlope = 'false'
elif (controlVars.variableEstimated=='tmin') & (len(controlVars.defaultTempLapse)!=0):
    tempDefaultLapse = Dataset(controlVars.defaultTempLapse)['tminLapse'][:]
    #override user specified choice for the recomputeDefaultTempSlope option
    parameters.recomputeDefaultTempSlope = 'false'
else: #else set the temp default lapse rate to spatially constant parameter value
    tempDefaultLapse = np.ones((grid.nc,grid.nr))*parameters.defaultSlope


In [18]:
tempDefaultLapse

array([[1.3, 1.3, 1.3, ..., 1.3, 1.3, 1.3],
       [1.3, 1.3, 1.3, ..., 1.3, 1.3, 1.3],
       [1.3, 1.3, 1.3, ..., 1.3, 1.3, 1.3],
       ...,
       [1.3, 1.3, 1.3, ..., 1.3, 1.3, 1.3],
       [1.3, 1.3, 1.3, ..., 1.3, 1.3, 1.3],
       [1.3, 1.3, 1.3, ..., 1.3, 1.3, 1.3]])

In [19]:
#check to see if there are any invalid slopes in the default temperature
#slope if used
if len(controlVars.defaultTempLapse)!=0:
    tempDefaultLapse[tempDefaultLapse < parameters.minSlope] = parameters.minSlope
    tempDefaultLapse[(tempDefaultLapse > parameters.maxSlopeLower) & (grid.layerMask == 1)] = parameters.maxSlopeLower
    tempDefaultLapse[(tempDefaultLapse > parameters.maxSlopeUpper) & (grid.layerMask == 2)] = parameters.maxSlopeUpper

    #set metGrid default slope to QC'ed spatial lapse 
    metGrid.defaultSlope = tempDefaultLapse


In [20]:
np.shape(metGrid.defaultSlope )

(136, 128)

# getNearStations:
* **finds nearby stations for current grid point**

In [21]:
def getNearStations(staLat,staLon,staFacet,gridLat,gridLon,gridFacet,nMatch,maxDist):

    """
     getNearStations finds nearby stations for current grid point

     Arguments:

      Input:   yPt, integer, y counter for current grid point
               xPt, integer, x counter for current grid point
               staLat, float, vector of station lat
               staLon, float, vector of station lon
               staFacet, integer, vector of station integer facets
               grid, structure, structure containing grid
               nMatch, integer, value of maximum number of stations to search for
               maxDist, float, value of maximum radius to search for stations

      Output:  nearStations, structure, structure containing indicies for stations
               within search radius and those within that search
               area on the same topographic facet as the current grid point


        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    
    class structtype():
        pass
    
    nearStations = structtype()
    
    
    lat1, lon1 = gridLat, gridLon
    lat2, lon2 = staLat, staLon
    radius = 6371  # km

    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = (np.sin(dlat / 2) * np.sin(dlat / 2) +
         np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) *
         np.sin(dlon / 2) * np.sin(dlon / 2))
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    staDist = radius * c
    staAngles =360+ (np.arctan(dlon/dlat)*180/np.pi)


    #get station indices from sorted distance (nearMatch)
    distSort= np.argsort(staDist, axis=None)
    nearMatch = distSort[0:int(nMatch)]

    #find stations on same topographic facet as current grid point
    staFacet= np.array(staFacet, dtype='float')
    matchFacet = np.where(staFacet == gridFacet)
    #get indicies of stations on same topographic facet
    matchFacetSort= np.argsort(staDist[matchFacet], axis=None) 


    #take nMatch stations on same facet with consideration of distance
    #from grid point
    stationInds = np.array(matchFacet)[0,matchFacetSort[0:int(nMatch)]]
    #now cull list based on distance from grid point
    matchDist = staDist[stationInds] <= maxDist
    #finalize station indices
    facetStationInds = stationInds[matchDist]

    #set output structure
    nearStations.staDist = staDist
    nearStations.staAngles = staAngles
    nearStations.nearStationInds = nearMatch
    nearStations.facetStationInds = facetStationInds
       
    
    return nearStations

# calcCoastWeights:
* **computes weights for stations as compared to the current grid point based on the differences in coastal distance**

In [22]:
def calcCoastWeights(gridDistanceToCoast,stationDistanceToCoast,coastalExp):

    """
     calcCoastWeights computes weights for stations as compared to the 
      current grid point based on the differences in coastal distance

     Arguments:

      Input: gridDistanceToCoast, float, distance to coast of current grid point
       stationDistanceToCoast, float, distance to coast for nearby stations

      Output:coastWeights, float, vector holding coastal weights for nearby stations

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    #define a tiny float
    tiny = 1e-5

    #distance to coast weighting (e.g. Daly et al. 2002)
    coastWeights = 1.0/((abs(gridDistanceToCoast-stationDistanceToCoast)+tiny)**(coastalExp))
    #check for values > 1
    coastWeights[coastWeights>1] = 1
    #normalize weights
    coastWeights = coastWeights/np.sum(coastWeights)
    
    return coastWeights

# calcTopoPositionWeights:
* **computes weights for nearby stations as compared to the current grid point based on the differences in topographic**

In [23]:
def calcTopoPositionWeights(gridTopoPosition,topoPosMinDiff,topoPosMaxDiff,topoPosExp,stationTopoPos):
    """"
    calcTopoPositionWeights computes weights for nearby stations as compared 
      to the current grid point based on the differences in topographic 
      position as in Daly et al. (2007), JAMC

     Arguments:

      Input:gridTopoPosition, float, distance to coast of current grid point
       stationTopoPos, float, vector of topographic position of nearby stations
       nearStationInds, integer, vector of nearby station indicies

      Output: topoPositionWeights, float, vector holding topographic position weights
                                   for nearby stations

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    #initialize topographic position vector
    topoPositionWeights = np.zeros([len(stationTopoPos),1])
    #compute topographic position difference
    topoDiff = abs(stationTopoPos-gridTopoPosition)

    for i in range(0,len(stationTopoPos)):

        #check if any differences are below min, set to 1 weight
        if topoDiff[i]<=topoPosMinDiff:
            topoPositionWeights[i] = 1
        #if differences are in between max and min, compute using linear decay
        elif topoDiff[i]>topoPosMinDiff:
            topoPositionWeights[i]=1/((topoDiff[i]/topoPosMinDiff)**topoPosExp)
        #if differences are larger than max, set to zero
        elif topoDiff[i]>topoPosMaxDiff:
            topoPositionWeights[i] = 0

    topoPositionWeights = np.divide(topoPositionWeights,np.sum(topoPositionWeights))
   
    return topoPositionWeights

# calcLayerWeights:
* *computes the weights of stations to the current grid point**

In [25]:
def calcLayerWeights(gridLayer,gridElev,stationLayer,stationElev,layerExp):
    """
    calcLayerWeights computes the weights of stations to the current grid point.  
    Estimates a 2-layer atmosphere (inversion and free) based on topography.  
    Helpful identifying inversion areas as defined in Daly et al. (2002, Climate Res.), section 7

     Arguments:

      Input: gridLayer, float, grid layer for current grid point
               gridElev, float, grid elevation for current grid point
               stationLayer, integer, layer of nearby stations
               stationElev, float, elevation of nearby stations
               layerExp, float, TIER parameter; exponent in weighting function

      Output: layerWeights, float, vector holding layer weights for nearby stations

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.



    """
    #define a tiny float
    tiny = 1e-6

    #find nearby stations that match the grid layer
    layerMatch=[]
    for i in range(len(stationLayer)):
        layerMatch.append(int(stationLayer[i])==gridLayer)
        
    #initalize weight variable
    layerWeights = np.zeros([len(stationLayer),1]) 
    
    
    #set station weight in same layer to 1
    layerWeights[layerMatch] = 1
    #compute weights for stations in other layer, based on vertical distance difference
    layerWeights[[not x for x in layerMatch]]= np.reshape(1/((abs(gridElev-np.array(stationElev)\
                                              [[not x for x in layerMatch]]+1)+tiny)**layerExp),(-1,1))
    
    #no station in other layer can have a weight >= 1.0
    layerWeights[layerWeights>=1] = 0.99
    #normalize weights
    layerWeights = layerWeights/np.sum(layerWeights)
    
    return layerWeights

# calcSymapWeights:
* *computes weights for nearby stations following the concept of the SYMAP algorithm**

In [26]:
def calcSymapWeights(staDist,staAngles,distanceWeightScale,distanceWeightExp,maxDist):
    """
     calcSymapWeights computes weights for nearby stations following the
      concept of the SYMAP algorithm (Shepard 1984).  Uses Barnes (1964) type
      distance weights instead of the exact SYMAP method.  Angular weighting
      is from Shepard (1984).

     Arguments:

      Input: staDist, float, vector of station distances to current grid point
       staAngles, float, vector of station angles from current grid point
       distanceWeightScale, float, input TIER parameter
       maxDist, float, input TIER parameter of maximum search distance

      Output: symapWeights,float, vector holding SYMAP weights of nearby stations

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """


    #set number of stations
    nMatch = len(staDist)

    #to radians
    toRad = np.pi/180.0

    #compute mean from nearest N stations using Barnes (1964) type distance weights
    #Set the scale factor to depends on mean distance of nearby stations
    if len(staDist):
        meanDist = mean(staDist)
        scale = distanceWeightScale*(meanDist/(maxDist));

        #compute initial distance weights
        distanceWeights = np.exp(-((staDist**distanceWeightExp)/scale))

        #directional isolation (Shepard 1984)
        #set angular isolation weight variable
        angleWeight = np.zeros([int(nMatch),1])
        #run through stations compute angle and isolation from other stations
        angleWeight=[]
        for i in range(0,nMatch):
            cosThetaSta = np.zeros([int(nMatch),1])
            cosThetaSta=[]
            for j in range(0,nMatch):
                #angle of station in radians from next station
                cosThetaSta.append(np.cos((staAngles[i]-staAngles[j])*toRad))

            #total angular isolation weight        
            angleWeight.append(np.sum(distanceWeights[i]*(1-np.array(cosThetaSta))))


        #increment integer vector: 1 to nMatch 
        inds = range(0,nMatch)
        angleWeight=np.array(angleWeight)
        
        # Supress/hide the warning
        np.seterr(invalid='ignore')

        #compute weights as a combination of distance and directional isolation
        symapWeights = np.multiply(distanceWeights**2,1+(angleWeight/np.sum(angleWeight[np.setxor1d(inds,i)])))
        #normalize weights
        symapWeights = symapWeights/np.sum(symapWeights)
    else:

        symapWeights= np.zeros([1,int(nMatch)]) 

    
    return symapWeights

# calcFinalWeights:
* *computes the weights of stations to the current grid point**

In [27]:
def calcFinalWeights(varEstimated,symapWeights,coastWeights,topoPositionWeights,layerWeights):
    """
     calcLayerWeights computes the weights of stations to the current grid
      point.  Estimates a 2-layer atmosphere (inversion and free) based on
      topography.  Helpful identifying inversion areas as defined in 
      Daly et al. (2002, Climate Res.), section 7

     Arguments:

      Input: varEstimated, string, met variable being estimated
       symapWeights, float, vector of SYMAP weights
       coastWeights, float, vector of distance to coast weights
       topoPositionWeights, float, vector of topographic position weights
       layerWeights, float, vector of inversion layer weights

      Output: finalWeights, float, vector holding final weights for nearby stations

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    #inversion layer and topographic position weighting (layerWeights 
    #and topoPosition) not used for precipitation
    if varEstimated=='precip':
        finalWeights = symapWeights*coastWeights
    elif (varEstimated=='tmax') or (varEstimated=='tmin'):
        finalWeights = symapWeights*coastWeights*topoPositionWeights*layerWeights
    else:
        print('error:Unrecognized variable: %s'%varEstimated)
    
    #normalize final weights
    finalWeights = finalWeights/np.sum(finalWeights)

    #prevent badly conditioned weight matrices in the regression set minium weight to a small 
    #number that is still large enough for numerics to compute
    finalWeights[finalWeights<1e-6] = 1e-6
    finalWeights[finalWeights==np.nan] = 1e-6 ####MAF:aded this shoild check later with andy
    
    return finalWeights

In [35]:
#######there is (x,y) that facetStationInds=[]########


# staLat= inputStations.meta.lat
# staLon= inputStations.meta.lon
# staFacet= inputStations.meta.facet
# gridLat=grid.lat[131,9]
# gridLon=grid.lon[131,9]
# gridFacet=grid.facet[131,9]
# nMatch=parameters.nMaxNear
# maxDist=parameters.maxDist


# lat1, lon1 = gridLat, gridLon
# lat2, lon2 = staLat, staLon
# radius = 6371  # km

# dlat = np.radians(lat2 - lat1)
# dlon = np.radians(lon2 - lon1)
# a = (np.sin(dlat / 2) * np.sin(dlat / 2) +
#      np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) *
#      np.sin(dlon / 2) * np.sin(dlon / 2))
# c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
# staDist = radius * c
# staAngles =np.arctan(dlon/dlat) * 180 / np.pi


# #get station indices from sorted distance (nearMatch)
# distSort= np.argsort(staDist, axis=None)
# nearMatch = distSort[0:int(nMatch)]

# #find stations on same topographic facet as current grid point
# staFacet= np.array(staFacet, dtype='float')
# matchFacet = np.where(staFacet == gridFacet)
# #get indicies of stations on same topographic facet
# matchFacetSort= np.argsort(staDist[matchFacet], axis=None) 


# #take nMatch stations on same facet with consideration of distance
# #from grid point
# stationInds = np.array(matchFacet)[0,matchFacetSort[0:int(nMatch)]]
# #now cull list based on distance from grid point
# matchDist = staDist[stationInds] <= maxDist
# #finalize station indices
# facetStationInds = stationInds[matchDist]


# calcWeightedRegression:
* *computes a weighted linear regression of station data against elevationn*

In [28]:
def calcWeightedRegression(stationElev,stationVar,stationWeights):
    """

     calcWeightedRegression computes a weighted linear regression of station
      data against elevation.  Weight vector determines weight of each station
      in the regression.  Here they are computed as the combination of the
      component geophysical weights (e.g. coastal, layer, SYMAP).  Following
      Daly et al. (1994,2002)
    
     Arguments:
    
      Input:
    
      stationElev, float, vector of elevation of nearby stations
       stationDist, float, vector of distance to nearby stations
       stationVar, float, vector containing meteorological variable values from
                      stations
       stationWeights, float,vector of station weights
    
      Output:
       
       linearFit, float, vector holding the two coefficients of the weighted
                         linear regression
    
     Author: Andrew Newman, NCAR/RAL
     Email : anewman@ucar.edu
     Postal address:
         P.O. Box 3000
         Boulder,CO 80307
    
     Copyright (C) 2019 University Corporation for Atmospheric Research
    
    This file is part of TIER.
    
     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.
    
     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.
    
     You should have received a copy of the GNU General Public License
    along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    
    #create weighted linear regression matricies
    n = len(stationElev)
    X = np.c_[np.ones(len(stationElev)), stationElev]
    W = np.eye(n,n)
    W[W==1] = stationWeights
    xw=X.T@W

    #compute weighted linear regression
    linearFit=(np.linalg.pinv(xw @ X) @ xw @ stationVar).T

    #change order of coefficients for convenience
    linearFit=np.roll(linearFit,-1)[0]

                 
    return linearFit

# calcPrecip:
* *computes the first pass TIER estimate of precipitation*

In [29]:
def calcPrecip(parameters,gridElev,defaultSlope,finalWeightsNear,finalWeightsFacet,symapWeights,\
               stationElevNear,stationElevFacet,stationVarNear,stationVarFacet):
    
    """
     calcPrcp computes the first pass TIER estimate of precipitation

     Summary: This algorithm generally follows Daly et al. (1994,2002,2007,2008) and
     others.  However, here the regression estimated parameters
     and the grid point elevation to compute the precipitation is not done.
     Instead, the baseInterp estimate is used at all grid points as the intercept
     and the weighted linear regression slope is used to adjust the baseInterp
     estimate up or down based on the elevation difference between the grid
     and the baseInterp weighted station elevation.  This approach gives similar
     results and is effectively an elevation adjustment to a baseInterp estimate
     where the baseInterp weights here are computed using all knowledge based
     terms in addition to the SYMAP distance & angular isolation weights
     Other modifications from the above cited papers are present and
     summarized below.


     Specific modifications/notes for initial precipitation implementation 
     (eq. 2, Daly et al. 2002):
     1) Here there are only 4-directional facets and flat (see tierPreprocessing.m for details)
        1 = N
        2 = E
        3 = S
        4 = W
        5 = Flat
     2) no elevation weighting factor
     3) cluster weighting factor using symap type weighting
     4) no topographic facet weighting factor

     5) number of stations and facet weighting:
     the nMaxNear nearest stations are always used to determine the base
     precip value, then only stations on the correct facet are
     used to determine the elevation-variable relationship (slope).  
     facet weighting where stations on same facet but with other facets 
     inbetween get less weight is not considered.  All stations on the same 
     facet get the same "facet" weight
     6) default lapse rates are updated after first pass to remove the
     spatially constant default lapse rate constraint (updatePrecipSlope.m)

     Arguments:

      Input:

       parameters  , structure, structure holding all TIER parameters
       gridElev    , float    , elevation of current grid point
       defaultSlope, float    , default normalized precipitation slope at
                                current grid point as current grid point
       finalWeights,       float, station weights for nearby stations
       finalWeightsFacet, float, station weights for nearby stations on same
                                  slope facet
       symapWeights,      float , symap station weights for nearby stations
       stationElevNear,   float , station elevations for nearby stations
       stationElevFacet, float , station elevations for nearby stations on
                                  same slope facet as current grid point
       stationVarNear   , float , station values for nearby stations
       stationVarFacet , float , sataion values for nearby stations on same 
                                  slope facet as current grid point

      Output:

       metPoint, structure, structure housing all grids related to
                            precipitation for the current grid point


        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.



    """
    #define tiny
    tiny = 1e-15
    #define large;
    large = 1e15

    #set local min station parameter
    nMinNear = parameters.nMinNear

    #local slope values
    minSlope = parameters.minSlope
    maxSlope = parameters.maxInitialSlope

    #initalize metPoint structure
    metPoint.rawField             = np.nan
    metPoint.intercept            = np.nan
    metPoint.slope                = np.nan
    metPoint.normSlope            = np.nan
    metPoint.baseInterpField      = np.nan
    metPoint.baseInterpElev       = np.nan
    metPoint.baseInterpUncert     = np.nan
    metPoint.slopeUncert          = np.nan
    metPoint.normSlopeUncert      = np.nan
    metPoint.intercept            = np.nan
    metPoint.validRegress         = np.nan

    #first estimate the 'baseInterp' value at grid point, but use full knowledge based weights if possible. 
    #this serves as the base estimate with no weighted linear elevation regression

    #if the final weight vector is invalid, default to symap weights only
    if np.isnan(finalWeightsNear[0]):
        #compute baseInterp precipitaiton
        metPoint.baseInterpField = np.sum([a*b for a,b in zip(symapWeights,stationVarNear)])/np.sum(symapWeights)
        #compute mean elevation of baseInterp stations
        metPoint.baseInterpElev = np.sum([a*b for a,b in zip(symapWeights,stationElevNear)])/np.sum(symapWeights)
        #estimate uncertainty using standard deviation of leave-one-out estimates
        nsta = len(symapWeights)
        combs = np.array(list(combinations(range(0, nsta), nsta-1)))

        symcombs=[symapWeights[combs[i,:]] for i in range(len(symapWeights))]
        varnearcombs= [stationVarNear[combs[i,:]][:,0] for i in range(len(stationVarNear))]
        metPoint.baseInterpUncert = np.std(np.sum([a*b for a,b in zip(symcombs,varnearcombs)], axis=1)/np.sum(symcombs, axis=1))
    else: #estimate simple average using final weights
        metPoint.baseInterpField = np.sum([a*b for a,b in zip(finalWeightsNear,stationVarNear)])/np.sum(finalWeightsNear)   
        metPoint.baseInterpElev = np.sum([a*b for a,b in zip(finalWeightsNear,stationElevNear)])/np.sum(finalWeightsNear)
        #estimate uncertainty using standard deviation of leave-one-out
        #estimates
        nsta = len(finalWeightsNear)
        combs = np.array(list(combinations(range(0, nsta), nsta-1)))

        finalcombs=[finalWeightsNear[combs[i,:]] for i in range(len(finalWeightsNear))]
        varnearcombs= [stationVarNear[combs[i,:]][:,0] for i in range(len(stationVarNear))]       
        metPoint.baseInterpUncert = np.std(np.sum([a*b for a,b in zip(finalcombs,varnearcombs)],axis=1)/np.sum(finalcombs,axis=1))

    #if there are more than nMinNear stations, proceed with weighted elevation regression
    if len(stationElevFacet) >= nMinNear:
        #create weighted linear regression relationship
        linFit = calcWeightedRegression(stationElevFacet,stationVarFacet,finalWeightsFacet);
        #define normalized slope estimate
        elevSlope = linFit[0]/np.mean(stationVarFacet)

        #Run through station combinations and find outliers to see if we can get a valid met var - elevation slope
        if (elevSlope < minSlope) or (elevSlope > maxSlope):
            #number of stations considered on current grid point facet
            nSta = len(stationVarFacet)
            #variable to track slope changes when estimating slope in weighted regression
            maxSlopeDelta = 0

            #set combinations for the number of combinations possbile selecting nSta-1 stations given nSta stations
            combs = np.array(list(combinations(range(0, nSta), nSta-1)))
            #counter to track number of valid slopes
            cnt = 1
            #initalize variable to hold valid slopes
            combSlp = []
            #step through combinations
            for c in range(len(combs[:,0])):
                #define station attributes matrix (X) for regression 
                X = np.c_[np.ones(len(np.array(stationElevFacet)[combs[c,:]])), np.array(stationElevFacet)[combs[c,:]]]

                #if X is square
                if np.shape(X)[0]==np.shape(X)[1]:
                    #if well conditioned
                    if (np.linalg.cond(X))**-1 >tiny:
                        #compute weighted regression
                        tmpLinFit = calcWeightedRegression(np.array(stationElevFacet)[combs[c]],np.array(stationVarFacet)[combs[c]]\
                                                           ,np.array(finalWeightsFacet)[combs[c]])
                        #set normalized slope variable
                        elevSlopeTest = tmpLinFit[0]/np.mean(np.array(stationVarFacet)[combs[c]])
                        #compute change in slope from initial estimate
                        slopeDelta = abs(elevSlope - elevSlopeTest)
                    else: #if X not well conditioned, set to unrealistic values
                        elevSlopeTest = large
                        slopeDelta = -large

                else:
                    #compute weighted regression
                    tmpLinFit = calcWeightedRegression(np.array(stationElevFacet)[combs[c]],np.array(stationVarFacet)[combs[c]]\
                                                           ,np.array(finalWeightsFacet)[combs[c]])
                    #set normalized slope variable
                    elevSlopeTest = tmpLinFit[0]/np.mean(np.array(stationVarFacet)[combs[c]])
                    #compute change in slope from initial estimate
                    slopeDelta = abs(elevSlope - elevSlopeTest)


                #if the recomputed slope is valid and has the largest slope
                #change, this specific case is the best estimate removing
                #the largest outlier in terms of estimated slope
                if (elevSlopeTest > minSlope) and (elevSlopeTest < maxSlope) and (slopeDelta > maxSlopeDelta):
                    removeOutlierInds = combs[c]
                    maxSlopeDelta = slopeDelta
                    #add valid slope to vector
                    combSlp.append(elevSlopeTest)
                    #increment counter
                    cnt = cnt + 1
                #catch the rest of the recomputed slopes that are valid
                elif (elevSlopeTest > minSlope) and (elevSlopeTest < maxSlope):
                    #add valid slope to vector
                    combSlp.append(elevSlopeTest)
                    #increment counter
                    cnt = cnt + 1


            #if two or more valid combination of stations estimate uncertainty of slope at grid point using standard
            #deviation of estimates
            if (cnt-1) >= 2:
                metPoint.normSlopeUncert = np.std(combSlp)               

            #if there was a valid outlier removal combination, use the 'best' combination
            if maxSlopeDelta>0:
                #compute regression estimate
                linFit = calcWeightedRegression(np.array(stationElevFacet)[removeOutlierInds],\
                                                np.array(stationVarFacet[removeOutlierInds]),\
                                                np.array(finalWeightsFacet[removeOutlierInds]))

                #override the regression intercept with the baseInterp estimate
                linFit[1] = metPoint.baseInterpField

                if np.isnan(linFit[0]):
                    linFit[0] = defaultSlope*metPoint.baseInterpField
                    tmpField = np.polyval(linFit,gridElev-metPoint.baseInterpElev)
                else:
                    tmpField = np.polyval(linFit,gridElev-metPoint.baseInterpElev)

                metPoint.rawField = tmpField
                metPoint.slope = linFit[0]
                metPoint.normSlope = linFit[0]/np.mean(stationVarFacet[removeOutlierInds])
                metPoint.intercept = linFit[1]
                metPoint.validRegress = 1
            else:
                linFit=np.zeros([2,1])
                linFit[0] = defaultSlope*metPoint.baseInterpField
                linFit[1] = metPoint.baseInterpField

                metPoint.rawField = np.polyval(linFit,gridElev-metPoint.baseInterpElev)
                metPoint.normSlope  = linFit[0]/np.mean(stationVarFacet)
                metPoint.slope      = linFit[0]
                metPoint.intercept  = linFit[1]

        #if regression coefficients are invalid
        elif np.isnan(linFit[0]):
            linFit=np.zeros([2,1])
            linFit[0] = defaultSlope*metPoint.baseInterpField
            linFit[1] = metPoint.baseInterpField

            metPoint.rawField  = np.polyval(linFit,gridElev-metPoint.baseInterpElev)
            metPoint.slope     = linFit[0]
            metPoint.normSlope = linFit[0]
            metPoint.intercept = linFit[1]
        else:

            linFit[1] = metPoint.baseInterpField

            metPoint.rawField = np.polyval(linFit,gridElev-metPoint.baseInterpElev)
            metPoint.slope     = linFit[0]
            metPoint.intercept = linFit[1]
            metPoint.normSlope = linFit[0]/np.mean(stationVarFacet)
            metPoint.validRegress = 1

            #run through station combinations to estimate uncertainty in
            #slope estimate
            nSta = len(stationVarFacet)
            combs = np.array(list(combinations(range(0, nSta), nSta-1)))
            cnt = 1


            combSlp = []
            #step through combinations
            for c in range(len(combs[:,0])):
                X = np.c_[np.ones(len(np.array(stationElevFacet)[combs[c,:]])), np.array(stationElevFacet)[combs[c,:]]]          

                #if X is square
                if np.shape(X)[0]==np.shape(X)[1]: 
                    #if X is well conditioned
                    if (np.linalg.cond(X))**-1 >tiny:
                        tmpLinFit = calcWeightedRegression(np.array(stationElevFacet)[combs[c]],np.array(stationVarFacet)[combs[c]]\
                                                           ,np.array(finalWeightsFacet)[combs[c]])
                        elevSlopeTest = tmpLinFit[0]/np.mean(np.array(stationVarFacet)[combs[c]])
                    else: #if not well conditioned, set variables to unrealistic values
                        elevSlopeTest = large

                else:
                    tmpLinFit = calcWeightedRegression(np.array(stationElevFacet)[combs[c]],np.array(stationVarFacet)[combs[c]]\
                                                           ,np.array(finalWeightsFacet)[combs[c]])
                    elevSlopeTest = tmpLinFit[0]/np.mean(np.array(stationVarFacet)[combs[c]])


                if (elevSlopeTest > minSlope) and (elevSlopeTest < maxSlope):
                    combSlp.append(elevSlopeTest)
                    cnt = cnt + 1



            #if two or more valid combination of stations estimate uncertainty of slope at grid point using standard
            #deviation of estimates
            if cnt-1 >= 2:
                metPoint.normSlopeUncert = np.std(combSlp)


    #only one station within range on Facet - revert to nearest with default slope
    elif len(stationVarFacet) == 1: 
        linFit=np.zeros([2,1])
        linFit[0] = defaultSlope*metPoint.baseInterpField
        linFit[1] = metPoint.baseInterpField

        metPoint.rawField = np.polyval(linFit,gridElev-metPoint.baseInterpElev)
        metPoint.slope     = linFit[0]
        metPoint.intercept = linFit[1]
        metPoint.normSlope = linFit[0]/metPoint.baseInterpField
    #less than nMinNear stations within range - attempt regression anyway
    elif (len(stationVarFacet) < nMinNear) and (len(stationVarFacet)):

        linFit = calcWeightedRegression(stationElevFacet,stationVarFacet,finalWeightsFacet)

        elevSlope = linFit[0]/np.mean(stationVarFacet)

        if (elevSlope < minSlope) or (elevSlope > maxSlope) or (np.isnan(elevSlope)):
            linFit[0] = defaultSlope*metPoint.baseInterpField
            linFit[1] = metPoint.baseInterpField

            metPoint.normSlope = linFit[0]/metPoint.baseInterpField
            metPoint.slope     = linFit[0]
            metPoint.intercept = linFit[1]
        else:

            metPoint.intercept = metPoint.baseInterpField
            metPoint.slope = linFit[0]
            metPoint.normSlope = linFit[0]/metPoint.baseInterpField


        #override intercept
        linFit[1] = metPoint.baseInterpField

        metPoint.rawField = np.polyval(linFit,gridElev-metPoint.baseInterpElev)

    else: #no original stations in distance search...
        linFit=np.zeros([2,1])
        linFit[0] = defaultSlope*metPoint.baseInterpField
        linFit[1] = metPoint.baseInterpField

        metPoint.rawField = np.polyval(linFit,gridElev-metPoint.baseInterpElev)
        metPoint.slope     = linFit[0]
        metPoint.intercept = linFit[1]
        metPoint.normSlope = linFit[1]/metPoint.baseInterpField

    
    return metPoint


# calcTemp:
* *computes the first pass TIER estimate of varEstimated*

In [30]:
def calcTemp(parameters,gridElev,defaultSlope,gridLayer,finalWeightsNear,finalWeightsFacet,symapWeights,\
             stationElevNear,stationElevFacet,stationVarNear,stationVarFacet):
    """
    Summary: This algorithm generally follows Daly et al. (1994,2002,2007,2008) and
     others.  However, here the regression estimated parameters
     and the grid point elevation to compute the precipitation is not done.
     Instead, the baseInterp estimate is used at all grid points as the intercept
     and the weighted linear regression slope is used to adjust the baseInterp
     estimate up or down based on the elevation difference between the grid
     and the baseInterp weighted station elevation.  This approach gives similar
     results and is effectively an elevation adjustment to a baseInterp estimate
     where the baseInterp weights here are computed using all knowledge based
     terms in addition to the SYMAP distance & angular isolation weights
     Other modifications from the above cited papers are present and
     summarized below.


     Specific modifications/notes for initial temperature implementation 
     (eq. 2, Daly et al. 2002):
     1) Here there are only 4-directional facets and flat (see tierPreprocessing.m for details)
        1 = N
        2 = E
        3 = S
        4 = W
        5 = Flat
     2) no elevation weighting factor
     3) cluster weighting factor using symap type weighting
     4) no topographic facet weighting factor

     5) number of stations and facet weighting:
     the nMaxNear nearest stations are always used to determine the base
     temperature value, then only stations on the correct facet are
     used to determine the elevation-variable relationship (slope).  
     facet weighting where stations on same facet but with other facets 
     inbetween get less weight is not considered.  All stations on the same 
     facet get the same "facet" weight
     6) default lapse rates can be determined from available sounding data that
     has been interpolated to the DEM
     7) used a search radius of 20-km for layer-1 or 2 determination for temp
     inversions (see tierPreprocessing.m for details)
     8) The coastal proximity weighting of Daly et al. (2003) is not
     implemented
     9) default lapse rates are updated after first pass to remove the
     spatially constant default lapse rate constraint (updateTempSlope.m)

     Arguments:

      Input:

       parameters  , structure, structure holding all TIER parameters
       gridElev    , float    , elevation of current grid point
       defaultSlope, float    , default slope for current grid point
       gridLayer   , float    , grid point layer in conceptual two-layer
                                atmosphere (Daly et al. 2002)
       finalWeights,       float, station weights for nearby stations
       finalWeightsFacet, float, station weights for nearby stations on same
                                  slope Facet
       symapWeights,      float , symap station weights for nearby stations
       stationElevNear,   float , station elevations for nearby stations
       stationElevFacet, float , station elevations for nearby stations on
                                  same slope Facet as current grid point
       stationVarNear   , float , station values for nearby stations
       stationVarFacet , float , sataion values for nearby stations on same 
                                  slope Facet as current grid point

      Output:

       metPoint, structure, structure housing all grids related to
                            precipitation for the current grid point

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.


    """
    #define tiny
    tiny = 1e-15
    #define large;
    large = 1e15
    
    #set local min station parameter
    nMinNear = parameters.nMinNear
    
    #local slope values
    minSlope = parameters.minSlope
    maxSlopeLower = parameters.maxSlopeLower
    maxSlopeUpper = parameters.maxSlopeUpper

    
    #initalize metPoint structure
    metPoint.rawField             = np.nan
    metPoint.intercept            = np.nan
    metPoint.slope                = np.nan
    metPoint.baseInterpField      = np.nan
    metPoint.baseInterpElev       = np.nan
    metPoint.baseInterpUncert     = np.nan
    metPoint.slopeUncert          = np.nan
    metPoint.intercept            = np.nan
    metPoint.validRegress         = np.nan
    
    #first estimate the 'baseInterp' value at grid point, but use full knowledge
    #based weights if possible. this serves as the base estimate with no 
    #weighted linear elevation regression


    #if the final weight vector is invalid, default to symap weights only
    if np.isnan(finalWeightsNear[0]):
        #compute baseInterp precipitaiton
        metPoint.baseInterpField = np.sum([a*b for a,b in zip(symapWeights,stationVarNear)])/np.sum(symapWeights)
        #compute mean elevation of baseInterp stations
        metPoint.baseInterpElev = np.sum([a*b for a,b in zip(symapWeights,stationElevNear)])/np.sum(symapWeights)
        #estimate uncertainty using standard deviation of leave-one-out estimates
        nsta = len(symapWeights)
        combs = np.array(list(combinations(range(0, nsta), nsta-1)))
                
        symcombs=[symapWeights[combs[i,:]] for i in range(len(symapWeights))]
        varnearcombs= [stationVarNear[combs[i,:]][:,0] for i in range(len(stationVarNear))]
        metPoint.baseInterpUncert = np.std(np.sum([a*b for a,b in zip(symcombs,varnearcombs)],\
                                                  axis=1)/np.sum(symcombs, axis=1))

    else: #estimate simple average using final weights
        metPoint.baseInterpField = np.sum([a*b for a,b in zip(finalWeightsNear,stationVarNear)])/np.sum(finalWeightsNear) 
        metPoint.baseInterpElev = np.sum([a*b for a,b in zip(finalWeightsNear,stationElevNear)])/np.sum(finalWeightsNear)
        #estimate uncertainty using standard deviation of leave-one-out estimates
        nsta = len(finalWeightsNear)
        combs = np.array(list(combinations(range(0, nsta), nsta-1)))

        finalcombs=[finalWeightsNear[combs[i,:]] for i in range(len(finalWeightsNear))]
        varnearcombs= [stationVarNear[combs[i,:]][:,0] for i in range(len(stationVarNear))]       
        metPoint.baseInterpUncert = np.std(np.sum([a*b for a,b in zip(finalcombs,varnearcombs)],axis=1)/np.sum(finalcombs,axis=1))



    #if there are more than nMinNear stations, proceed with weighted elevation regression
    if len(stationElevFacet) >= nMinNear:
        #create weighted linear regression relationship
        linFit = calcWeightedRegression(stationElevFacet,stationVarFacet,finalWeightsFacet)

        #Run through station combinations and find outliers to see if we can get a valid met var - elevation slope
        #make comparisons for valid slope temperature has bounds for each layer
        elevSlope = linFit[0]
        if (elevSlope < minSlope) or (elevSlope > maxSlopeLower and gridLayer==1) or\
                (elevSlope>maxSlopeUpper and gridLayer==2):
                
            #number of stations considered on current grid point facet
            nSta = len(stationVarFacet)
            #variable to track slope changes when estimating slope in weighted regression
            maxSlopeDelta = 0
                
            for i in nSta-1:
                combs = np.array(list(combinations(range(0, nSta), i)))
                cnt = 1
                combSlp = []
                for c in range(len(combs[:,0])):
                    X = np.c_[np.ones(len(np.array(stationElevFacet)[combs[c,:]])), np.array(stationElevFacet)[combs[c,:]]]

                    #if X is square
                    if np.shape(X)[0]==np.shape(X)[1]:
                        #if X is well conditioned
                        if (np.linalg.cond(X))**-1 >tiny:
                            tmpLinFit = calcWeightedRegression(np.array(stationElevFacet)[combs[c]],np.array(stationVarFacet)[combs[c]]\
                                                           ,np.array(finalWeightsFacet)[combs[c]])
                            elevSlopeTest = tmpLinFit[0]
                            slopeDelta = abs(elevSlope - elevSlopeTest)
                        else: #if X not well conditioned, set to unrealistic values
                            elevSlopeTest = large
                            slopeDelta = -large
                        
                    else:
                        tmpLinFit = calcWeightedRegression(np.array(stationElevFacet)[combs[c]],np.array(stationVarFacet)[combs[c]]\
                                                           ,np.array(finalWeightsFacet)[combs[c]])
                        elevSlopeTest = tmpLinFit[0]
                        slopeDelta = abs(elevSlope - elevSlopeTest)
                    

                    if elevSlopeTest > minSlope and slopeDelta > maxSlopeDelta:
                        if (elevSlopeTest < maxSlopeLower and gridLayer == 1) or\
                            (elevSlopeTest < maxSlopeUpper and gridLayer == 2):
                
                            removeOutlierInds = combs[c]
                            maxSlopeDelta = slopeDelta
                            combSlp.append(elevSlopeTest) 
                            cnt = cnt + 1
                        
                    elif elevSlopeTest > minSlope:
                        if (elevSlopeTest < maxSlopeLower and gridLayer == 1) or \
                            (elevSlopeTest < maxSlopeUpper and gridLayer == 2):
                            combSlp.append(elevSlopeTest)
                            cnt = cnt + 1
                                        
                #if two or more valid combination of stations estimate uncertainty of slope at grid point using standard
                #deviation of estimates
                if (cnt-1) >= 2:
                    metPoint.slopeUncert = np.std(combSlp)
                
            
            
            if maxSlopeDelta>0:
                linFit = calcWeightedRegression(np.array(stationElevFacet)[removeOutlierInds],\
                                                np.array(stationVarFacet[removeOutlierInds]),\
                                                np.array(finalWeightsFacet[removeOutlierInds]))
                
                                            
                linFit[1] = metPoint.baseInterpField
                
                if np.isnan(linFit[0]):
                    linFit[0] = defaultSlope
                    tmpField = np.polyval(linFit,gridElev-metPoint.baseInterpElev)
                else:
                    tmpField = np.polyval(linFit,gridElev-metPoint.baseInterpElev)
                
                metPoint.rawField = tmpField
                metPoint.slope = linFit[0]
                metPoint.intercept = linFit[1]
                metPoint.validRegress = 1
                
            else:
                linFit=np.zeros([2,1])
                linFit[0] = defaultSlope;
                linFit[1] = metPoint.baseInterpField
                
                metPoint.rawField = np.polyval(linFit,gridElev-metPoint.baseInterpElev);
                metPoint.slope      = linFit[0]
                metPoint.intercept  = linFit[1]
            

        elif np.isnan(linFit[0]):
            linFit=np.zeros([2,1])
            linFit[0] = defaultSlope
            linFit[1] = metPoint.baseInterpField
            
            metPoint.rawField  = np.polyval(linFit,gridElev-metPoint.baseInterpElev)
            metPoint.slope     = linFit[0]
            metPoint.intercept = linFit[1]
        else:           
            
            linFit[1] = metPoint.baseInterpField
            
            metPoint.rawField = np.polyval(linFit,gridElev-metPoint.baseInterpElev)
            metPoint.slope     = linFit[0]
            metPoint.intercept = linFit[1]
            metPoint.validRegress = 1

            #run through station combinations to estimate uncertainty in
            #slope estimate
            nSta = len(stationVarFacet)
            combs = np.array(list(combinations(range(0, nSta), nSta-1)))
            cnt = 1
            combSlp = []
                             
                
            for c in range(len(combs[:,0])):
                X = np.c_[np.ones(len(np.array(stationElevFacet)[combs[c,:]])), np.array(stationElevFacet)[combs[c,:]]]
                
                #if X is square
                if np.shape(X)[0]==np.shape(X)[1]:
                    #if X is well conditioned
                    if (np.linalg.cond(X))**-1 >tiny:
                        tmpLinFit = calcWeightedRegression(np.array(stationElevFacet)[combs[c]],np.array(stationVarFacet)[combs[c]]\
                                                           ,np.array(finalWeightsFacet)[combs[c]])
                        elevSlopeTest = tmpLinFit[0]
                    else:
                        elevSlopeTest = large
                    
                else:
                    tmpLinFit = calcWeightedRegression(np.array(stationElevFacet)[combs[c]],np.array(stationVarFacet)[combs[c]]\
                                                           ,np.array(finalWeightsFacet)[combs[c]])
                    elevSlopeTest = tmpLinFit[0]
                
                
                if(elevSlopeTest < maxSlopeLower and elevSlopeTest > minSlope and gridLayer == 1)\
                    or (elevSlopeTest<maxSlopeUpper and elevSlopeTest>minSlope and gridLayer == 2):
                    combSlp.append(elevSlopeTest)
                    cnt = cnt + 1
                            
            #if two or more valid combination of stations estimate uncertainty of slope at grid point using standard
            #deviation of estimates
            if (cnt-1) >= 2:
                metPoint.slopeUncert = np.std(combSlp)
            

    else: #not enough stations within range on Facet - revert to nearest with default slope
        linFit=np.zeros([2,1])
        linFit[0] = defaultSlope
        linFit[1] = metPoint.baseInterpField
        
        metPoint.rawField = np.polyval(linFit,gridElev-metPoint.baseInterpElev)
        metPoint.slope     = linFit[0]
        metPoint.intercept = linFit[1]

     
    return metPoint

In [31]:
class structtype():
    pass
nearStations = structtype()
coastWeights = structtype()
topoPositionWeights = structtype()
layerWeights = structtype()
symapWeights = structtype()
finalWeights = structtype()
metPoint = structtype()

#loop through all grid points and perform regression
for y in range(0,grid.nr):   
    for x in range(0,grid.nc):
        if grid.mask[x,y] > 0:
            
            #find nearby stations to current grid point
            nearStations = getNearStations(inputStations.meta.lat,inputStations.meta.lon,inputStations.meta.facet,\
                                           grid.lat[x,y],grid.lon[x,y],grid.facet[x,y],parameters.nMaxNear,\
                                           parameters.maxDist)

            #compute coastal distance weights
            stationDistanceToCoast_near= [inputStations.meta.coastDist[i] for i in nearStations.nearStationInds]
            coastWeights.near = calcCoastWeights(grid.distToCoast[x,y],stationDistanceToCoast_near,parameters.coastalExp)

            stationDistanceToCoast_facet= [inputStations.meta.coastDist[i] for i in nearStations.facetStationInds]
            coastWeights.facet = calcCoastWeights(grid.distToCoast[x,y],stationDistanceToCoast_facet,parameters.coastalExp)

            #compute topographic position weights
            stationTopoPos_near= [inputStations.meta.topoPosition[i] for i in nearStations.nearStationInds]
            topoPositionWeights.near = calcTopoPositionWeights(grid.topoPosition[x,y],parameters.topoPosMinDiff,\
                                                               parameters.topoPosMaxDiff,parameters.topoPosExp,\
                                                               stationTopoPos_near)

            stationTopoPos_facet= [inputStations.meta.topoPosition[i] for i in nearStations.facetStationInds]
            topoPositionWeights.facet = calcTopoPositionWeights(grid.topoPosition[x,y],parameters.topoPosMinDiff,\
                                                                parameters.topoPosMaxDiff,parameters.topoPosExp,\
                                                                stationTopoPos_facet)
            #compute layer weights
            stationLayer_near = [inputStations.meta.layer[i] for i in nearStations.nearStationInds]
            stationElev_near = [inputStations.meta.elev[i] for i in nearStations.nearStationInds]
            layerWeights.near = calcLayerWeights(grid.layerMask[x,y],grid.dem[x,y],stationLayer_near,stationElev_near,\
                                                 parameters.layerExp)

            stationLayer_facet = [inputStations.meta.layer[i] for i in nearStations.facetStationInds]
            stationElev_facet = [inputStations.meta.elev[i] for i in nearStations.facetStationInds]
            layerWeights.facet = calcLayerWeights(grid.layerMask[x,y],grid.dem[x,y],stationLayer_facet,stationElev_facet,\
                                                  parameters.layerExp)


            #compute SYMAP weights
            symapWeights.near = calcSymapWeights(nearStations.staDist[nearStations.nearStationInds],\
                                                 nearStations.staAngles[nearStations.nearStationInds],\
                                                 parameters.distanceWeightScale,parameters.distanceWeightExp,\
                                                 parameters.maxDist)

            symapWeights.facet = calcSymapWeights(nearStations.staDist[nearStations.facetStationInds],\
                                                  nearStations.staAngles[nearStations.facetStationInds],\
                                                  parameters.distanceWeightScale,parameters.distanceWeightExp,\
                                                  parameters.maxDist)

            #compute final weights
            finalWeights.near = calcFinalWeights(controlVars.variableEstimated,symapWeights.near,coastWeights.near,\
                                                 topoPositionWeights.near,layerWeights.near)
            finalWeights.facet = calcFinalWeights(controlVars.variableEstimated,symapWeights.facet,coastWeights.facet,\
                                                  topoPositionWeights.facet,layerWeights.facet)



            #compute first pass met field on grid
            if controlVars.variableEstimated=='precip':
                #compute met fields at current grid point for precipitation
                stationElevFacet = [inputStations.meta.elev[i] for i in nearStations.facetStationInds]
                stationElevNear = [inputStations.meta.elev[i] for i in nearStations.nearStationInds]
                metPoint = calcPrecip(parameters,grid.smoothDemKM[x,y],parameters.defaultSlope,finalWeights.near,\
                                      finalWeights.facet,symapWeights.near,stationElevNear,stationElevFacet,\
                                      inputStations.avgVar[nearStations.nearStationInds],\
                                      inputStations.avgVar[nearStations.facetStationInds])


                #set precipitation specific output variables
                metGrid.normSlopeUncert[x,y] = metPoint.normSlopeUncert
                metGrid.normSlope[x,y]       = metPoint.normSlope

            elif (controlVars.variableEstimated=='tmax') or (controlVars.variableEstimated=='tmin'):
                #compute met fields at current grid point for temperature
                metPoint = calcTemp(parameters,grid.smoothDemKM[x,y],tempDefaultLapse[x,y],grid.layerMask[x,y],\
                                    finalWeights.near,finalWeights.facet,symapWeights.near,\
                                   [inputStations.meta.elev[i] for i in nearStations.nearStationInds],\
                                    [inputStations.meta.elev[i] for i in nearStations.facetStationInds],\
                                    inputStations.avgVar[nearStations.nearStationInds],\
                                    inputStations.avgVar[nearStations.facetStationInds])


            #set metGrid values for current grid point
            metGrid.rawField[x,y]            = metPoint.rawField
            metGrid.intercept[x,y]           = metPoint.intercept
            metGrid.slope[x,y]               = metPoint.slope
            metGrid.baseInterpField[x,y]     = metPoint.baseInterpField
            metGrid.baseInterpElev[x,y]      = metPoint.baseInterpElev
            metGrid.baseInterpUncert[x,y]    = metPoint.baseInterpUncert
            metGrid.slopeUncert[x,y]         = metPoint.slopeUncert
            metGrid.validRegress[x,y]        = metPoint.validRegress

   


In [32]:
def fspecial_gauss(size, sigma):

    """
    Function to mimic the 'fspecial' gaussian MATLAB function
    """

    x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()

# updatePrecipSlope:
* *updates the estimated slope (elevation lapse rate) of precipitation across the grid from the initial estimate*

In [33]:
def updatePrecipSlope(nr,nc,mask,normSlope,validSlope,defaultSlope,recomputeDefault,filterSize,filterSpread):
    """
     updatePrecipSlope updates the estimated slope (elevation lapse rate) of 
     precipitation across the grid from the initial estimate

     Arguments:

      Inputs: nr, integer,   number of rows in grid
       nc, integer,   number of columns in grid
       mask, integer, mask of valid grid points
       normslope, float, intial normalized slope estimate across grid
       validSlope, integer, mask of valid regression estimated slopes
       defaultSlope, float, value of the default normalized precipitation
                            slope 
       recomputeDefault,string, string indicating true/false to recompute the
                            default normalized precipitation slope
       filterSize, integer, size of low-pass filter in grid points
       filterSpread, float, variance of low-pass filter

      Outputs: finalNormSlope, structure, structure containing the final normalized 
                                  slope for all grid points for precip variables

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu


     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    #if user specifies to recompute the default slope then do it use only points that had valid regression based slopes
    #ideally this is an improvement over a specified default slope
    
    if recomputeDefault=='true':
        baseSlope = normSlope
        baseSlope[validSlope!=1] = -999
        domainMeanSlope = np.mean(np.mean(baseSlope[baseSlope != -999]))
        baseSlope[baseSlope == -999] = domainMeanSlope
    else:
        baseSlope = normSlope
        baseSlope[validSlope!=1] = defaultSlope


    #filter and interpolate slopes to entire domain 
    #this is a similar step to the Daly et al. (1994) feathering except it is applied to the precipitation slopes.
    #This is done before feathering, where feathering is a final check for precipitation gradients

    #define a mesh of indicies for scattered interpolation of valid points back to a grid
    x = range(0,nc)
    y = range(0,nr)

    x2d,y2d = np.meshgrid(y,x)

    #find valid grid points
    j,i=np.where(baseSlope > 0)
    #scattered interpolation using griddata
    interpBaseSlope = griddata((i,j),baseSlope[baseSlope>0],(x2d,y2d),'linear')


    #fill missing values with the average of elements
    interpBaseSlope= np.nan_to_num(interpBaseSlope, nan=np.nanmean(interpBaseSlope))
    #interpBaseSlope = fillmissing(interpBaseSlope,'nearest',2)


    #define gaussian low-pass filter
    gFilter = fspecial_gauss(filterSize,filterSpread)

    #filter slope estimate
    filterSlope= ndimage.convolve(interpBaseSlope, gFilter, mode='wrap', cval=0)


    #set unused grid points to missing
    filterSlope[mask<=0] = -999

    #set output variable
    finalNormSlope = filterSlope
    
    
    return finalNormSlope


# featherPrecip:
* *updates the estimated precipitation field to remove sharp, potentially unrealistic gradients due primarily do to slope facet processing*

In [34]:
def featherPrecip(parameters,nr,nc,dx,dem,mask,finalNormSlope,baseInterpPrecip,baseInterpElev):
    """
     featherPrecip updates the estimated precipitation field to remove sharp, potentially unrealistic gradients 
     due primarily do to slope facet processing. Generally follows Daly et al.(1994).  
     This is the final precipitation processing step.

     Arguments:

      Inputs: parameters, structure, tier parameter structure
       dxy, float, grid spacing (km) of grid
       dem, float, grid dem
       mask, integer, mask of valid grid points
       finalNormslope, float, final normalized slope estimate across grid
       baseInterpPrecip, float,    baseInterp weighted estimated precipitation 
       baseInterpElev, float, elevation of baseInterp weighted stations for baseInterp estimate

      Outputs: finalPrecip, float, grid containing the final precipitation estimate

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    
    """
    #compute precipitation again, using spatially interpolated valid slopes,
    #rather than a mishmash of valid and default values from grid point to grid point
    finalPrecip = np.multiply(np.multiply(finalNormSlope,baseInterpPrecip),(dem-baseInterpElev)) + baseInterpPrecip
    finalSlope = np.multiply(finalNormSlope,baseInterpPrecip)

    #Now check to see if any feathering is needed to relax potentially large grid cell to grid-cell gradients
    #that may be non-physical.  generally follows Daly et al. (1994)

    #Here a check of the grid elevation is made to prevent excessive
    #feathering across low and flat elevations.  This may be because of
    #some bug in the algorithm, or some other unknown issue.  This check
    #does not appear to be in Daly et al. (1994)



    #set parameter values from input parameter structure
    #buffer to add to slope change to make sure gradient falls under max_grad when changed
    bufferSlope = parameters.bufferSlope 
    maxFinalSlope = parameters.maxFinalSlope  #max normalized slope (Daly et al. 1994)
    maxGrad = parameters.maxGrad  #normalized slope per km  #each grid cell is dx apart
    demMaxGrad = maxGrad/dx.filled() #grid relative gradient
    minElevDiff = parameters.minElevDiff #km
    minElev = parameters.minElev #km

    #first, set negative and zero pcp to mean value
    negInds = np.where(finalPrecip<=0)
    if np.size(negInds)!=0:
        finalPrecip[negInds] = baseInterpPrecip[negInds]


    #second need to make sure all the precipitation slopes are within reasonable bounds
    inds = np.where(finalNormSlope>maxFinalSlope)
    if np.size(inds)!=0:
        #reset values to valid final maximum slope
        finalSlope[inds] = maxFinalSlope*baseInterpPrecip[inds]

        #recompute precipitation field using updated slopes
        for i in  range(len(inds)):
            finalPrecip[inds[i]] = np.polyval(np.c_[finalSlope[inds],baseInterpPrecip[inds]][:,i],dem[inds][i]-baseInterpElev[inds][i])


    #now check spatial gradients for exceedance
    #recompute normalized slopes in a temporary variable
    tmpNormSlope = np.divide(finalSlope,baseInterpPrecip)
    #temporary precipitation variable
    tmpPrecip = finalPrecip.filled(np.nan)
    tmpNormSlope[mask<0]=np.nan

    #keep track of passes and number of grid points modified
    pass1 = 1
    numgridModified = 1
    while numgridModified > 0:
        numgridModified = 0
        #loop across grid
        for x in range(1,nr-1):
            if np.mod(x,100)==0:
                print('Done with pass %d, row: %d\n'%(pass1,y))

            for y in range(1,nc-1):

                #compute gradients using trailing pt (backward finite difference)

                #two step process: East-West graident first
                ewGrad = tmpNormSlope[y,x]-tmpNormSlope[y-1,x]

                #if the gradient exceeds the allowable gradient and the elevation is above minimum and the difference
                #across grid cell elevations is large enough then feather grid point
                if (abs(ewGrad)> demMaxGrad) and ((abs(dem[y,x]- dem[y-1,x])>minElevDiff) or (dem[y,x]>minElev)):
                    #define static fields
                    tmpIntercept = [baseInterpPrecip[y,x], baseInterpPrecip[y-1,x]]
                    tmpElev = [baseInterpElev[y,x], baseInterpElev[y-1,x]]
                    tmpDem = [dem[y,x], dem[y-1,x]]

                    #define fields that will be updated
                    tmpPrecipArray = [tmpPrecip[y,x], tmpPrecip[y-1,x]]
                    tmpSlopeArray = [tmpNormSlope[y,x], tmpNormSlope[y-1,x]]


                    #which point has smaller precipitation
                    lowInd = np.argmin(tmpPrecipArray)

                    #if the slope of the lower precipitation estimate is valid
                    if tmpSlopeArray[lowInd]>=maxFinalSlope:
                        #find the index of the minimum slope point
                        lowInd = np.argmin(tmpSlopeArray);

                        #feather (smooth) the slope gradient using the valid slope
                        tmpPtSlope = ((abs(ewGrad)-demMaxGrad)+bufferSlope+tmpSlopeArray[lowInd])
                        #recompute precipitation with smoothed slope


                        tmpPtPrecip = np.polyval(np.c_[tmpPtSlope*tmpIntercept[lowInd],tmpIntercept[lowInd]],\
                                                 tmpDem[lowInd]-tmpElev[lowInd])

                        #update the precipitation and slope values
                        tmpPrecip[y-lowInd,x] = tmpPtPrecip
                        tmpNormSlope[y-lowInd,x] = tmpPtSlope
                    else:
                        #if the slope of the lower precipitation estimate is not valid
                        #feather the gradient using lower precipitation grid point slope.  
                        #this will eventually smooth gradients out across enough grid points given enough passes
                        tmpPtSlope = ((abs(ewGrad)-demMaxGrad)+bufferSlope+tmpSlopeArray[lowInd])
                        tmpPtPrecip = np.polyval(np.c_[tmpPtSlope*tmpIntercept[lowInd],tmpIntercept[lowInd]],\
                                                 tmpDem[lowInd]-tmpElev[lowInd])

                        #update precipitation and slope values
                        tmpPrecip[y-lowInd,x] = tmpPtPrecip
                        tmpNormSlope[y-lowInd,x] = tmpPtSlope
                    # end temporary slope check if statement 

                    #increment counter tracking number of point modifications
                    numgridModified = numgridModified + 1
                 #end precipitation gradient check

                #North-South second using updated precipitation and slope 
                #if the East-West gradient feathering changed things
                #compute North-South gradient
                nsGrad = tmpNormSlope[y,x]-tmpNormSlope[y,x-1]


                #if the gradient exceeds the allowable gradient
                #and the elevation is above minimum and the difference
                #across grid cell elevations is large enough
                #then feather grid point
                if abs(nsGrad)>demMaxGrad and ((abs(dem[y,x]-dem[y,x-1])>minElevDiff) or (dem(y,x)>minElev)):
                    #set static fields
                    tmpIntercept = [baseInterpPrecip[y,x], baseInterpPrecip(y,x-1)]
                    tmpElev      = [baseInterpElev[y,x], baseInterpElev(y,x-1)]
                    tmpDem       = [dem[y,x], dem[y,x-1]]

                    #set updated fields
                    tmpPrecipArray = [tmpPrecip[y,x], tmpPrecip[y,x-1]]
                    tmpSlopeArray = [tmpNormSlope[y,x], tmpNormSlope[y,x-1]]

                    #which point is lower precipitation
                    lowInd = np.argmin(tmpPrecipArray)

                    #if the slope of the lower precipitation estimate is valid
                    if tmpSlopeArray[lowInd]>=maxFinalSlope:
                        #find the index of the minimum slope point
                        lowInd = np.argmin(tmpSlopeArray)
                        #feather (smooth) the slope gradient using the valid slope
                        tmpPtSlope = ((abs(nsGrad)-demMaxGrad)+bufferSlope+tmpSlopeArray[lowInd])
                        tmpPtPrecip = np.polyval(np.c_[tmpPtSlope*tmpIntercept[lowInd],tmpIntercept[lowInd]],\
                                                 tmpDem[lowInd]-tmpElev[lowInd])
                        #recompute precipitation with smoothed slope
                        tmpPrecip[y,x-lowInd+1] = tmpPtPrecip
                        tmpNormSlope[y,x-lowInd+1] = tmpPtSlope
                    else:
                        #if the slope of the lower precipitation estimate is not valid
                        #feather the gradient using lower precipitation grid point slope.  
                        #this will eventually smooth gradients out across enough grid points given enough passes
                        tmpPtSlope = ((abs(nsGrad)-demMaxGrad)+bufferSlope+tmpSlopeArray[lowInd])
                        tmpPtPrecip = np.polyval(np.c_[tmpPtSlope*tmpIntercept[lowInd],tmpIntercept[lowInd]],\
                                                       tmpDem[lowInd]-tmpElev[lowInd])

                        #update precipitation and slope values
                        tmpPrecip[y,x-lowInd+1] = tmpPtPrecip
                        tmpNormSlope[y,x-lowInd+1] = tmpPtSlope

                    #increment counter tracking number of point
                    #modifications
                    numgridModified = numgridModified + 1

                 #gradient check

             #x
         #y
        #increment pass counter
        pass1 = pass1 + 1;
        print('%d points modified\n'%numgridModified)
      #passes


    #finally recheck any negative and zero pcp to mean value
    negInds = np.where(tmpPrecip<=0)
    if np.size(negInds)!=0:
        tmpPrecip[negInds] = baseInterpPrecip[negInds]

    #set final precipitation
    finalPrecip = tmpPrecip
    #invalid grid points set to missing
    finalPrecip[mask<0] = -999

    return finalPrecip


# calcFinalPrecipUncert:
* *produces the final component uncertainty estimates as well as the final total and relative uncertainty accounting for covariance between the components of the total*

In [35]:
def calcFinalPrecipUncert(grid,baseInterpUncert,baseInterpElev,slopeUncert,finalVar,filterSize,filterSpread,covWindow):
    
    """
     calcFinalPrecipUncert produces the final component uncertainty estimates as well as the final total 
     and relative uncertainty accounting for covariance between the components of the total

     Arguments:

      Inputs: grid, structure, structure containing grid information
       baseInterpUncert, float, baseInterp precipitation uncertainty estimate (mm timestep-1)
       slopeUncert, float, estimated uncertainty of slope (elev lapse rate) in normalized space
       finalVar, float, final variable estimate (precip here)
       filterSize, integer, size of low-pass filter in grid points
       filterSpread, float, variance of low-pass filter
       covWindow, float, size (grid points) of covariance window

      Outputs: finalUncert, structure, structure containing the final components
               and the total and relative precipitation uncertainty estimates

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.
    
    """
    #use only points that had valid uncertainty estimates from the base
    #baseInterp interpolation or the weighted regression, then filter and interpolate to entire domain 
    #this estimates uncertainty at each grid point from points where we
    #actually have initial estimates also smooths out high-frequency noise


    class structtype():
        pass
    finalUncert = structtype()    

    #define a mesh of indicies for scattered interpolation of valid points back to a grid
    y = range(0,grid.nr)
    x = range(0,grid.nc)
    x2d,y2d = np.meshgrid(y,x)

    #find valid baseInterpUncert points
    j,i=np.where(baseInterpUncert > 0)

    #scattered interpolation using griddata
    interpBaseInterp =griddata((i,j),baseInterpUncert[baseInterpUncert>=0],(x2d,y2d),'linear')
    #fill missing values with nearest neighbor

    interpBaseInterp = np.nan_to_num(interpBaseInterp, nan=np.nanmean(interpBaseInterp))

    #find valid slopeUncert points
    j,i = np.where(slopeUncert >= 0) 

    #scattered interpolation using griddata
    interpSlope = griddata((i,j),slopeUncert[slopeUncert>=0],(x2d,y2d),'linear')

    #fill missing values with nearest neighbor
    interpSlope = np.nan_to_num(interpSlope, nan=np.nanmean(interpSlope))


    #generate gaussian low-pass filter
    gFilter = fspecial_gauss(filterSize,filterSpread)


    #filter uncertainty estimates
    finalBaseInterpUncert = ndimage.convolve(interpBaseInterp, gFilter, mode='wrap', cval=0)
    finalSlopeUncert = ndimage.convolve(interpSlope, gFilter, mode='wrap', cval=0)


    #estimate the total and relative uncertainty in physical units (mm timestep-1)
    #compute slope in physical space
    baseSlopeUncert = np.multiply(np.multiply(finalSlopeUncert,finalVar),abs(baseInterpElev-(grid.smoothDem/1000))) #need to have dem in km

    baseSlopeUncert= np.nan_to_num(baseSlopeUncert, nan=np.nanmean(baseSlopeUncert))


    #replace nonvalid mask points with NaN
    baseSlopeUncert[grid.mask<=0] = np.nan
    finalBaseInterpUncert[grid.mask<=0] = np.nan

    #define a local covariance vector
    localCov = np.zeros(np.shape(finalBaseInterpUncert))*np.nan

    #step through each grid point and estimate the local covariance between
    #the two uncertainty components using covWindow to define the size of the local covariance estimate
    #covariance influences the total combined estimate
    for i in range(grid.nr):
        for j in range(grid.nc):
            #define indicies aware of array bounds
            jInds = [max(0, i-covWindow),min(grid.nr,i+covWindow)]
            iInds = [max(0, j-covWindow),min(grid.nc,j+covWindow)]

            #compute local covariance using selection of valid points get windowed area
            subBaseInterp = finalBaseInterpUncert[int(iInds[0]):int(iInds[1]),int(jInds[0]):int(jInds[1])]
            subSlope = baseSlopeUncert[int(iInds[0]):int(iInds[1]),int(jInds[0]):int(jInds[1])]
            #compute covariance for only valid points in window
            c = np.cov(subBaseInterp[np.isnan(subBaseInterp)==False],subSlope[np.isnan(subSlope)==False])
            #pull relevant value from covariance matrix
            localCov[j,i] = c[0,len(c[0,:])-1]



    #compute the total estimates 
    finalUncert.totalUncert = baseSlopeUncert+finalBaseInterpUncert+2*np.sqrt(abs(localCov))
    finalUncert.relativeUncert = np.divide(finalUncert.totalUncert,finalVar)

    #set novalid gridpoints to missing 
    finalBaseInterpUncert[grid.mask<=0] = -999
    finalUncert.totalUncert[grid.mask<=0] = -999
    finalUncert.relativeUncert[grid.mask<=0] = -999

    #define components in output structure
    finalUncert.finalBaseInterpUncert = finalBaseInterpUncert
    finalUncert.finalSlopeUncert = baseSlopeUncert
   
    
    return finalUncert


# updateTempSlope:
* *updates the estimated slope (elevation lapse rate) of temperature variables across the grid from the initial estimate*

In [36]:
def updateTempSlope(nr,nc,mask,gridLayer,slope,recomputeDefault,defaultSlope,\
                    validSlope,minSlope,maxSlopeLower,maxSlopeUpper,filterSize,filterSpread):
    """
     updateTempSlope updates the estimated slope (elevation lapse rate) of temperature 
     variables across the grid from the initial estimate

     Arguments:

      Inputs: nr, integer,   number of rows in grid
       nc, integer,   number of columns in grid
       mask, integer, mask of valid grid points
       slope, float, intial slope estimate across grid
       defaultSlope, float, default estimate of slope across grid
       recomputeDefault,string, string indicating true/false to recompute the
       default normalized precipitation slope validSlope, integer, mask of valid regression estimated slopes
       minSlope, float, minimum valid slope (TIER parameter)
       maxSlopeLower, float, maximum lower layer valid slope (TIER parameter)
       maxSlopeUpper, float, maximum upper layer valid slope (TIER parameter)
       filterSize, integer, size of low-pass filter in grid points
       filterSpread, float, variance of low-pass filter

      Outputs: finalSlope, structure, structure containing the final slope for all
               grid points for temp variables

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    #if user specifies to recompute the default slope and there is
    #no user specified spatially variable lapse rate file (this check is performed in the driver)
    #use only points that had valid regression based slopes
    #ideally this is an improvement over a specified default slope
    if recomputeDefault=='true':
        baseSlope = slope
        baseSlope[validSlope!=1] = -999
        domainMeanSlope = np.mean(np.mean(baseSlope[baseSlope != -999]))
        baseSlope[baseSlope == -999] = domainMeanSlope
    else:
        baseSlope = slope
        if len(defaultSlope[:,0]>1):
            baseSlope[validSlope!=1] = defaultSlope[validSlope!=1]
        else:
            baseSlope[validSlope!=1] = defaultSlope
            


    #define a mesh of indicies for scattered interpolation of valid points back to a grid
    y = range(0,nr)
    x = range(0,nc)
    x2d,y2d = np.meshgrid(y,x)

    #perform 2 scattered interpolations to get final slope, one for layer 1, one for layer2
    
    #find valid points for layer 1
    j,i=np.where((baseSlope >= minSlope) & (gridLayer == 1))
    
    #scattered interpolation using griddata
    interpSlopeLayer1 = griddata((i,j),baseSlope[(baseSlope>= minSlope) & (gridLayer == 1)],(x2d,y2d),'linear')
    #fill missing values with nearest neighbor

    interpSlopeLayer1 = np.nan_to_num(interpSlopeLayer1, nan=np.nanmean(interpSlopeLayer1))
    
    #find valid points for layer 2
    [i,j] = np.where((baseSlope >= minSlope) & (gridLayer == 2))
    #scattered interpolation using griddata
    interpSlopeLayer2 = griddata((i,j),baseSlope[(baseSlope>= minSlope) & (gridLayer == 2)],(x2d,y2d),'linear')
    #fill missing values with nearest neighbor

    interpSlopeLayer2 = np.nan_to_num(interpSlopeLayer2, nan=np.nanmean(interpSlopeLayer2))
    
    #define gaussian low-pass filter
    gFilter = fspecial_gauss(filterSize,filterSpread)
    
    
    #filter the combined field
    filterSlope = interpSlopeLayer1
    filterSlope[gridLayer == 2] = interpSlopeLayer2[gridLayer == 2]

    filterSlope = ndimage.convolve(filterSlope, gFilter, mode='wrap', cval=0)
    

    #check for invalid slopes
    filterSlopeLayer1 = filterSlope
    filterSlopeLayer1[filterSlopeLayer1 > maxSlopeLower] = maxSlopeLower
    filterSlopeLayer1[filterSlopeLayer1 < minSlope] = minSlope
    #set unused points to missing
    filterSlopeLayer1[mask<=0] = -999
    
    #check for invalid slopes
    filterSlopeLayer2 = filterSlope
    filterSlopeLayer2[filterSlopeLayer2 > maxSlopeUpper] = maxSlopeUpper
    filterSlopeLayer2[filterSlopeLayer2 < minSlope] = minSlope
    #set unused points to missing
    filterSlopeLayer2[mask<=0] = -999
    
    #combine the two layer estimates into one complete grid
    finalSlope = filterSlopeLayer1
    finalSlope[gridLayer == 2] = filterSlopeLayer2[gridLayer == 2]

    return finalSlope


# calcFinalTemp:
* *computes the final temperature grid after all adjustmentse*

In [37]:
def calcFinalTemp(dem,mask,baseInterpElev,baseInterpTemp,finalSlope):
    """
     calFinalTemp computes the final temperature grid after all adjustments

     Arguments:

      Inputs: dem,  float  , grid dem
       mask, integer, mask of valid grid points

       baseInterpElev, float, elevation of baseInterp weighted stations for baseInterp estimate
       baseInterpTemp, float, baseInterp estimated temperature
       finaSlope, float, grid of final slope estimates after any previous adjustments 

      Outputs: finalTemp, structure, structure containing the final temperature
                             estimate across the grid

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    #compute final temp using all finalized estimates
    finalTemp = np.multiply(finalSlope,(dem-baseInterpElev) ) + baseInterpTemp
    #set unused grid points to missing
    finalTemp[mask<0] = -999
    
    return finalTemp


# calcFinalTempUncert:
* *computes the final uncertainty for temperature variables*

In [38]:
def calcFinalTempUncert(grid,baseInterpUncert,baseInterpElev,slopeUncert,filterSize,filterSpread,covWindow):
    """
     calcFinalTempUncert computes the final uncertainty for temperature variables

     Arguments:

      Inputs: grid, structure, structure containing grid information
       baseInterpUncert, float, intiial baseInterp uncertainty estimate across grid
       slopeUncert, float, initial estimate of slope uncertainty across grid
       filterSize, integer, size of low-pass filter in grid points
       filterSpread, float, variance of low-pass filter
       covWindow, float, size (grid points) of covariance window

      Outputs: finalUncert, structure, structure containing total and relative
                               uncertainty for met var

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.
    """
    
    #define a mesh of indicies for scattered interpolation of valid points back to a grid
    y = range(0,nr)
    x = range(0,nc)
    x2d,y2d = np.meshgrid(y,x)

    #find valid points for baseInterp uncertainty
    j,i = np.where(np.isnan(baseInterpUncert)== False)
    #scattered interpolation using griddata
    interpBaseInterp = griddata((i,j),baseInterpUncert[np.isnan(baseInterpUncert)==False],(x2d,y2d),'linear')
    #fill missing values with nearest neighbor
    interpBaseInterp = np.nan_to_num(interpBaseInterp, nan=np.nanmean(interpBaseInterp))  
    
    #find valid points for slope uncertaintty
    j,i = np.where(slopeUncert >= 0)
    #scattered interpolation using griddata
    interpSlope = griddata((i,j),slopeUncert[slopeUncert>=0],(x2d,y2d),'linear');
    interpSlope = np.multiply(interpSlope,abs(baseInterpElev-grid.smoothDem))
    #fill missing values with nearest neighbor

    interpSlope = np.nan_to_num(interpSlope, nan=np.nanmean(interpSlope)) 
    
    
    #gaussian low-pass filter
    gFilter =  fspecial_gauss(filterSize,filterSpread)
    
    #filter uncertainty estimates
    finalBaseInterpUncert = ndimage.convolve(interpBaseInterp, gFilter, mode='wrap', cval=0)
    finalSlopeUncert = ndimage.convolve(interpSlope, gFilter, mode='wrap', cval=0)

    
    #replace nonvalid mask points with NaN
    finalBaseInterpUncert[grid.mask<=0]= np.nan
    finalSlopeUncert[grid.mask<=0] = np.nan
    
    #estimate the total and relative uncertainty in physical units 
    #define a local covariance vector
    localCov =np.zeros(np.shape(finalBaseInterpUncert))*np.nan

    #step through each grid point and estimate the local covariance between
    #the two uncertainty components using covWindow to define the size of the local covariance estimate
    #covariance influences the total combined estimate
    for i in range(grid.nr):
        for j in range(grid.nc):
            #define indicies aware of array bounds
            jInds = [max(0, i-covWindow),min(grid.nr,i+covWindow)]
            iInds = [max(0, j-covWindow),min(grid.nc,j+covWindow)]

            #compute local covariance using selection of valid points
            #get windowed area
            subBaseInterp = finalBaseInterpUncert[int(iInds[0]):int(iInds[1]),int(jInds[0]):int(jInds[1])]
            subSlope = finalSlopeUncert[int(iInds[0]):int(iInds[1]),int(jInds[0]):int(jInds[1])]
            #compute covariance for only valid points in window
            c = np.cov(subBaseInterp[np.isnan(subBaseInterp)==False],subSlope[np.isnan(subSlope)==False])
            #pull relevant value from covariance matrix
            localCov[i,j] = c[0,len(c[0,:])-1]      
            
        
    

    #compute the total estimates 
    finalUncert.totalUncert = finalSlopeUncert+finalBaseInterpUncert+2*np.sqrt(abs(localCov))
    finalUncert.relativeUncert = np.zeros(np.shape(finalUncert.totalUncert))*np.nan

    #set novalid gridpoints to missing
    finalBaseInterpUncert[grid.mask<=0] = -999
    finalSlopeUncert[grid.mask<=0] = -999
    finalUncert.totalUncert[grid.mask<=0] = -999
    finalUncert.relativeUncert[grid.mask<=0] = -999   

    #define components in output structure
    finalUncert.finalBaseInterpUncert = finalBaseInterpUncert
    finalUncert.finalSlopeUncert = finalSlopeUncert
 
    
    return finalUncert


In [39]:
#update and compute final fields conditioned on met variable
if controlVars.variableEstimated=='precip':
    #re-compute slope estimate
    finalNormSlope = updatePrecipSlope(grid.nr,grid.nc,grid.mask,metGrid.normSlope,metGrid.validRegress,\
                                       parameters.defaultSlope,parameters.recomputeDefaultPrecipSlope,\
                                       parameters.filterSize,parameters.filterSpread)
    
    #compute final field value
    #feather precipitation generally following Daly et al. (1994)
    metGrid.finalField = featherPrecip(parameters,grid.nr,grid.nc,grid.dx,grid.smoothDemKM,grid.mask,finalNormSlope,\
                                       metGrid.baseInterpField,metGrid.baseInterpElev)
    
    #compute final uncertainty estimate
    finalUncert = calcFinalPrecipUncert(grid,metGrid.baseInterpUncert,metGrid.baseInterpElev,metGrid.normSlopeUncert,\
                                        metGrid.baseInterpField,parameters.filterSize,parameters.filterSpread,\
                                        parameters.covWindow)
    
    #set metGrid variables
    metGrid.finalSlope = finalNormSlope*metGrid.finalField
    metGrid.finalSlope[grid.mask<0] = -999 #set final slope value to missing where mask is ocean
    metGrid.totalUncert = finalUncert.totalUncert
    metGrid.relUncert = finalUncert.relativeUncert
    metGrid.baseInterpUncert = finalUncert.finalBaseInterpUncert
    metGrid.slopeUncert = finalUncert.finalSlopeUncert
    metGrid.defaultSlope = np.ones([grid.nc,grid.nr])*parameters.defaultSlope

elif (controlVars.variableEstimated=='tmax') or (controlVars.variableEstimated=='tmin'):
    #re-compute slope estimate
    metGrid.finalSlope = updateTempSlope(grid.nr,grid.nc,grid.mask,grid.layerMask,metGrid.slope,\
                                         parameters.recomputeDefaultTempSlope,metGrid.defaultSlope,metGrid.validRegress,\
                                         parameters.minSlope,parameters.maxSlopeLower,parameters.maxSlopeUpper,\
                                         parameters.filterSize,parameters.filterSpread)

    #compute final field estimate
    metGrid.finalField = calcFinalTemp(grid.smoothDemKM,grid.mask,metGrid.baseInterpElev,metGrid.baseInterpField,\
                                       metGrid.finalSlope)
    
    #compute final uncertainty estimate
    finalUncert = calcFinalTempUncert(grid,metGrid.baseInterpUncert,metGrid.baseInterpElev,metGrid.slopeUncert,\
                                      parameters.filterSize,parameters.filterSpread,parameters.covWindow)

    #set metGrid variables
    metGrid.totalUncert = finalUncert.totalUncert
    metGrid.relUncert = finalUncert.relativeUncert
    metGrid.baseInterpUncert = finalUncert.finalBaseInterpUncert
    metGrid.slopeUncert = finalUncert.finalSlopeUncert
    metGrid.defaultSlope = tempDefaultLapse



Done with pass 1, row: 134

0 points modified



C:\Users\askarzam\Anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\askarzam\AppData\Local\Temp/ipykernel_5312/1505896360.py:111: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = np.cov(subBaseInterp[np.isnan(subBaseInterp)==False],subSlope[np.isnan(subSlope)==False])
C:\Users\askarzam\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [45]:
outputName=controlVars.outputName
outputVar=controlVars.variableEstimated

#size of grid
nc,nr = np.shape(metGrid.rawField)

#units check
if outputVar=='precip':
    physicalUnits = 'mm/day'
    normSlopeUnits = 'km-1'
    slopeUnits = 'mm/km'
elif (outputVar=='tmax') or (outputVar=='tmin'):
    physicalUnits = 'deg_C'
    slopeUnits = 'deg_C/km'
    normSlopeUnits = 'undefined'

#check to see if output file name exists
#if it does remove it and rewrite
if os.path.isfile(outputName)== True:
    #create string for system call
    print('remove %s' %outputName)
    os.remove(outputName)
    
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass


#Save to netcdf file
#add grid fields first
#create file, set dimensions and write elevation
ncfile = Dataset(controlVars.outputName,mode='w',format='NETCDF4_CLASSIC') 
ncfile.createDimension('longitude',  nc)     
ncfile.createDimension('latitude',  nr)


# Create the attributes   of elevation  
elevation = ncfile.createVariable('elevation', np.float64,('longitude','latitude'), fill_value= -999.0)
elevation.setncattr('name','Domain elevation')
elevation.setncattr('long_name','Domain elevation')
elevation.setncattr('units','m')
# Appends data along unlimited dimension   
elevation[:,:] = grid.dem*1000 #convert back to m


latitude = ncfile.createVariable('latitude', np.float64,('longitude','latitude'), fill_value= -999.0)
latitude.setncattr('name','latitude')
latitude.setncattr('long_name','latitude')
latitude.setncattr('units','degrees_north')
latitude[:,:] = grid.lat


longitude= ncfile.createVariable('longitude', np.float64,('longitude','latitude'), fill_value= -999.0)
longitude.setncattr('name','longitude')
longitude.setncattr('long_name','longitude')
longitude.setncattr('units','degrees_west')
longitude[:,:] = grid.lon

mask= ncfile.createVariable('mask', np.float64,('longitude','latitude'), fill_value= -999.0)
mask.setncattr('name','Domain mask')
mask.setncattr('long_name','Mask that sets land (1, valid), ocean (-1, met values not computed), and inland lake (0, met values computed)')
mask.setncattr('units','-')
mask[:,:] = grid.mask

#now on to output variables from the TIER model
#rawField
rawField= ncfile.createVariable('rawField', np.float64,('longitude','latitude'), fill_value= -999.0)
rawField.setncattr('name','raw variable output')
rawField.setncattr('long_name','Raw variable output before slope and gradient adjustments')
rawField.setncattr('units',physicalUnits)
rawField[:,:] = metGrid.rawField

#intercept
intercept= ncfile.createVariable('intercept', np.float64,('longitude','latitude'), fill_value= -999.0)

intercept.setncattr('name','intercept parameter')
intercept.setncattr('long_name','Intercept parameter from the variable-elevation regression')
intercept.setncattr('units',physicalUnits)
intercept[:,:] = metGrid.intercept

#slope
slope= ncfile.createVariable('slope', np.float64,('longitude','latitude'), fill_value= -999.0)
slope.setncattr('name','variable elevation slope')
slope.setncattr('long_name','Raw variable elevation slope before slope adjustments')
slope.setncattr('units',slopeUnits)
slope[:,:] = metGrid.slope

#normalized slope (valid for precipitation only)
#normSlope
normSlope= ncfile.createVariable('normSlope', np.float64,('longitude','latitude'), fill_value= -999.0)
normSlope.setncattr('name','normalized variable slope')
normSlope.setncattr('long_name','normalized variable elevation slope before slope adjustments(valid for precipitation only)')
normSlope.setncattr('units',normSlopeUnits)
normSlope[:,:] = metGrid.normSlope

#baseInterpField
baseInterpField= ncfile.createVariable('baseInterpField', np.float64,('longitude','latitude'), fill_value= -999.0)
baseInterpField.setncattr('name','baseInterp estimate')
baseInterpField.setncattr('long_name','baseInterp estimated variable values on grid')
baseInterpField.setncattr('units',physicalUnits)
baseInterpField[:,:] = metGrid.baseInterpField

#baseInterpElev
baseInterpElev=ncfile.createVariable('baseInterpElev', np.float64,('longitude','latitude'), fill_value= -999.0)
baseInterpElev.setncattr('name','Weighted elevation')
baseInterpElev.setncattr('long_name','Grid point elevation estimate using station elevations and final weights')
baseInterpElev.setncattr('units','m')
baseInterpElev[:,:] = metGrid.baseInterpElev

#baseInterpUncert
baseInterpUncert= ncfile.createVariable('baseInterpUncert', np.float64,('longitude','latitude'), fill_value= -999.0)
baseInterpUncert.setncattr('name','baseInterp uncertainty')
baseInterpUncert.setncattr('long_name','Uncertainty estimate from the baseInterp variable estimate')
baseInterpUncert.setncattr('units',physicalUnits)
baseInterpUncert[:,:] = metGrid.baseInterpUncert


#slopeUncert
slopeUncert= ncfile.createVariable('slopeUncert', np.float64,('longitude','latitude'), fill_value= -999.0)
slopeUncert.setncattr('name','slope uncertainty')
slopeUncert.setncattr('long_name','Uncertainty estimate (physical space) resulting from the variable-elevation slope estimate')
slopeUncert.setncattr('units',physicalUnits)
slopeUncert[:,:] = metGrid.slopeUncert

#normSlopeUncert (valid for precipitation only)
normSlopeUncert = ncfile.createVariable('normSlopeUncert', np.float64,('longitude','latitude'), fill_value= -999.0)
normSlopeUncert.setncattr('name','normalized slope uncertainty')
normSlopeUncert.setncattr('long_name','Uncertainty estimate (normalized) resulting from the variable-elevation slope estimate(valid for precipitation only)')
normSlopeUncert.setncattr('units',normSlopeUnits)
baseInterpUncert[:,:] = metGrid.normSlopeUncert

#defaultSlope
defaultSlope= ncfile.createVariable('defaultSlope', np.float64,('longitude','latitude'), fill_value= -999.0)
defaultSlope.setncattr('name','default slope')
defaultSlope.setncattr('long_name','default elevation-variable slope estimate')
defaultSlope.setncattr('units',slopeUnits)
defaultSlope[:,:] = metGrid.defaultSlope

#finalSlope
finalSlope= ncfile.createVariable('finalSlope', np.float64,('longitude','latitude'), fill_value= -999.0)
finalSlope.setncattr('name','final slope')
finalSlope.setncattr('long_name','Final variable elevation slope after slope adjustments')
finalSlope.setncattr('units',slopeUnits)
finalSlope[:,:] = metGrid.finalSlope

#finalField
finalField= ncfile.createVariable('finalField', np.float64,('longitude','latitude'), fill_value= -999.0)
finalField.setncattr('name','final variable output')
finalField.setncattr('long_name','Final variable output after slope and gradient adjustments')
finalField.setncattr('units',physicalUnits)
finalField[:,:] = metGrid.finalField

#totalUncert
totalUncert= ncfile.createVariable('totalUncert', np.float64,('longitude','latitude'), fill_value= -999.0)
totalUncert.setncattr('name','total uncertainty')
totalUncert.setncattr('long_name','total uncertainty in physical units')
totalUncert.setncattr('units',physicalUnits)
totalUncert[:,:] = metGrid.totalUncert

#relUncert
relUncert= ncfile.createVariable('relUncert', np.float64,('longitude','latitude'), fill_value= -999.0)
relUncert.setncattr('name','relative uncertainty')
relUncert.setncattr('long_name','relative total uncertainty')
relUncert.setncattr('units','-')
relUncert[:,:] = metGrid.relUncert

#validRegress
validRegress=ncfile.createVariable('validRegress', np.float64,('longitude','latitude'), fill_value= -999.0)
validRegress.setncattr('name','valid regression')
validRegress.setncattr('long_name','flag denoting the elevation-variable regression produced a valid slope')
validRegress.setncattr('units','-')
validRegress[:,:] = metGrid.validRegress

#save parameters to output file as well
#write this out as global attributes
ncfile.nMaxNear=parameters.nMaxNear
ncfile.nMaxNear =parameters.nMaxNear
ncfile.nMinNear=parameters.nMinNear
ncfile.maxDist=parameters.maxDist
ncfile.minSlope=parameters.minSlope
ncfile.maxInitialSlope= parameters.maxInitialSlope
ncfile.maxFinalSlope=parameters.maxFinalSlope
ncfile.maxSlopeLower=parameters.maxSlopeLower
ncfile.maxSlopeUpper=parameters.maxSlopeUpper
ncfile.defaultSlope=parameters.defaultSlope
ncfile.topoPosMinDiff=parameters.topoPosMinDiff
ncfile.topoPosMaxDiff =parameters.topoPosMaxDiff
ncfile.topoPosExp =parameters.topoPosExp
ncfile.costalExp =parameters.coastalExp
ncfile.layerExp =parameters.layerExp
ncfile.distanceWeightScale =parameters.distanceWeightScale
ncfile.distanceWeightExp =parameters.distanceWeightExp
ncfile.maxGrad =parameters.maxGrad
ncfile.bufferSlope =parameters.bufferSlope
ncfile.minElev =parameters.minElev
ncfile.minElevDiff =parameters.minElevDiff
ncfile.recomputeDefaultPrecipSlope = parameters.recomputeDefaultPrecipSlope
ncfile.filterSize =parameters.filterSize
ncfile.filterSpread =parameters.filterSpread
ncfile.covWindow =parameters.covWindow



remove C:\Users\askarzam\Downloads\TIER-master\data\output\caliOutput_prcp_13sta_test.nc


# saveOutput:
* *saves the metGrid structure into a netcdf file*

In [46]:
def saveOutput(outputName,outputVar,grid,metGrid,parameters):
    """
     saveOutput saves the metGrid structure into a netcdf file

     Arguments:

      Input: outputName, string   , name of output file
       metGrid, structure, structure containing TIER met fields

      Output: None

        Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
        Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    #size of grid
    nc,nr = np.shape(metGrid.rawField)

    #units check
    if outputVar=='precip':
        physicalUnits = 'mm/day'
        normSlopeUnits = 'km-1'
        slopeUnits = 'mm/km'
    elif (outputVar=='tmax') or (outputVar=='tmin'):
        physicalUnits = 'deg_C'
        slopeUnits = 'deg_C/km'
        normSlopeUnits = 'undefined'

    #check to see if output file name exists
    #if it does remove it and rewrite
    if os.path.isfile(outputName)== True:
        #create string for system call
        print('remove %s' %outputName)
        os.remove(outputName)

    try: ncfile.close()  # just to be safe, make sure dataset is not already open.
    except: pass


    #Save to netcdf file
    #add grid fields first
    #create file, set dimensions and write elevation
    ncfile = Dataset(controlVars.outputName,mode='w',format='NETCDF4_CLASSIC') 
    ncfile.createDimension('longitude',  nc)     
    ncfile.createDimension('latitude',  nr)


    # Create the attributes   of elevation  
    elevation = ncfile.createVariable('elevation', np.float64,('longitude','latitude'), fill_value= -999.0)
    elevation.setncattr('name','Domain elevation')
    elevation.setncattr('long_name','Domain elevation')
    elevation.setncattr('units','m')
    # Appends data along unlimited dimension   
    elevation[:,:] = grid.dem*1000 #convert back to m


    latitude = ncfile.createVariable('latitude', np.float64,('longitude','latitude'), fill_value= -999.0)
    latitude.setncattr('name','latitude')
    latitude.setncattr('long_name','latitude')
    latitude.setncattr('units','degrees_north')
    latitude[:,:] = grid.lat


    longitude= ncfile.createVariable('longitude', np.float64,('longitude','latitude'), fill_value= -999.0)
    longitude.setncattr('name','longitude')
    longitude.setncattr('long_name','longitude')
    longitude.setncattr('units','degrees_west')
    longitude[:,:] = grid.lon

    mask= ncfile.createVariable('mask', np.float64,('longitude','latitude'), fill_value= -999.0)
    mask.setncattr('name','Domain mask')
    mask.setncattr('long_name','Mask that sets land (1, valid), ocean (-1, met values not computed), and inland lake (0, met values computed)')
    mask.setncattr('units','-')
    mask[:,:] = grid.mask

    #now on to output variables from the TIER model
    #rawField
    rawField= ncfile.createVariable('rawField', np.float64,('longitude','latitude'), fill_value= -999.0)
    rawField.setncattr('name','raw variable output')
    rawField.setncattr('long_name','Raw variable output before slope and gradient adjustments')
    rawField.setncattr('units',physicalUnits)
    rawField[:,:] = metGrid.rawField

    #intercept
    intercept= ncfile.createVariable('intercept', np.float64,('longitude','latitude'), fill_value= -999.0)

    intercept.setncattr('name','intercept parameter')
    intercept.setncattr('long_name','Intercept parameter from the variable-elevation regression')
    intercept.setncattr('units',physicalUnits)
    intercept[:,:] = metGrid.intercept

    #slope
    slope= ncfile.createVariable('slope', np.float64,('longitude','latitude'), fill_value= -999.0)
    slope.setncattr('name','variable elevation slope')
    slope.setncattr('long_name','Raw variable elevation slope before slope adjustments')
    slope.setncattr('units',slopeUnits)
    slope[:,:] = metGrid.slope

    #normalized slope (valid for precipitation only)
    #normSlope
    normSlope= ncfile.createVariable('normSlope', np.float64,('longitude','latitude'), fill_value= -999.0)
    normSlope.setncattr('name','normalized variable slope')
    normSlope.setncattr('long_name','normalized variable elevation slope before slope adjustments(valid for precipitation only)')
    normSlope.setncattr('units',normSlopeUnits)
    normSlope[:,:] = metGrid.normSlope

    #baseInterpField
    baseInterpField= ncfile.createVariable('baseInterpField', np.float64,('longitude','latitude'), fill_value= -999.0)
    baseInterpField.setncattr('name','baseInterp estimate')
    baseInterpField.setncattr('long_name','baseInterp estimated variable values on grid')
    baseInterpField.setncattr('units',physicalUnits)
    baseInterpField[:,:] = metGrid.baseInterpField

    #baseInterpElev
    baseInterpElev=ncfile.createVariable('baseInterpElev', np.float64,('longitude','latitude'), fill_value= -999.0)
    baseInterpElev.setncattr('name','Weighted elevation')
    baseInterpElev.setncattr('long_name','Grid point elevation estimate using station elevations and final weights')
    baseInterpElev.setncattr('units','m')
    baseInterpElev[:,:] = metGrid.baseInterpElev

    #baseInterpUncert
    baseInterpUncert= ncfile.createVariable('baseInterpUncert', np.float64,('longitude','latitude'), fill_value= -999.0)
    baseInterpUncert.setncattr('name','baseInterp uncertainty')
    baseInterpUncert.setncattr('long_name','Uncertainty estimate from the baseInterp variable estimate')
    baseInterpUncert.setncattr('units',physicalUnits)
    baseInterpUncert[:,:] = metGrid.baseInterpUncert


    #slopeUncert
    slopeUncert= ncfile.createVariable('slopeUncert', np.float64,('longitude','latitude'), fill_value= -999.0)
    slopeUncert.setncattr('name','slope uncertainty')
    slopeUncert.setncattr('long_name','Uncertainty estimate (physical space) resulting from the variable-elevation slope estimate')
    slopeUncert.setncattr('units',physicalUnits)
    slopeUncert[:,:] = metGrid.slopeUncert

    #normSlopeUncert (valid for precipitation only)
    normSlopeUncert = ncfile.createVariable('normSlopeUncert', np.float64,('longitude','latitude'), fill_value= -999.0)
    normSlopeUncert.setncattr('name','normalized slope uncertainty')
    normSlopeUncert.setncattr('long_name','Uncertainty estimate (normalized) resulting from the variable-elevation slope estimate(valid for precipitation only)')
    normSlopeUncert.setncattr('units',normSlopeUnits)
    baseInterpUncert[:,:] = metGrid.normSlopeUncert

    #defaultSlope
    defaultSlope= ncfile.createVariable('defaultSlope', np.float64,('longitude','latitude'), fill_value= -999.0)
    defaultSlope.setncattr('name','default slope')
    defaultSlope.setncattr('long_name','default elevation-variable slope estimate')
    defaultSlope.setncattr('units',slopeUnits)
    defaultSlope[:,:] = metGrid.defaultSlope

    #finalSlope
    finalSlope= ncfile.createVariable('finalSlope', np.float64,('longitude','latitude'), fill_value= -999.0)
    finalSlope.setncattr('name','final slope')
    finalSlope.setncattr('long_name','Final variable elevation slope after slope adjustments')
    finalSlope.setncattr('units',slopeUnits)
    finalSlope[:,:] = metGrid.finalSlope

    #finalField
    finalField= ncfile.createVariable('finalField', np.float64,('longitude','latitude'), fill_value= -999.0)
    finalField.setncattr('name','final variable output')
    finalField.setncattr('long_name','Final variable output after slope and gradient adjustments')
    finalField.setncattr('units',physicalUnits)
    finalField[:,:] = metGrid.finalField

    #totalUncert
    totalUncert= ncfile.createVariable('totalUncert', np.float64,('longitude','latitude'), fill_value= -999.0)
    totalUncert.setncattr('name','total uncertainty')
    totalUncert.setncattr('long_name','total uncertainty in physical units')
    totalUncert.setncattr('units',physicalUnits)
    totalUncert[:,:] = metGrid.totalUncert

    #relUncert
    relUncert= ncfile.createVariable('relUncert', np.float64,('longitude','latitude'), fill_value= -999.0)
    relUncert.setncattr('name','relative uncertainty')
    relUncert.setncattr('long_name','relative total uncertainty')
    relUncert.setncattr('units','-')
    relUncert[:,:] = metGrid.relUncert

    #validRegress
    validRegress=ncfile.createVariable('validRegress', np.float64,('longitude','latitude'), fill_value= -999.0)
    validRegress.setncattr('name','valid regression')
    validRegress.setncattr('long_name','flag denoting the elevation-variable regression produced a valid slope')
    validRegress.setncattr('units','-')
    validRegress[:,:] = metGrid.validRegress

    #save parameters to output file as well
    #write this out as global attributes
    ncfile.nMaxNear=parameters.nMaxNear
    ncfile.nMaxNear =parameters.nMaxNear
    ncfile.nMinNear=parameters.nMinNear
    ncfile.maxDist=parameters.maxDist
    ncfile.minSlope=parameters.minSlope
    ncfile.maxInitialSlope= parameters.maxInitialSlope
    ncfile.maxFinalSlope=parameters.maxFinalSlope
    ncfile.maxSlopeLower=parameters.maxSlopeLower
    ncfile.maxSlopeUpper=parameters.maxSlopeUpper
    ncfile.defaultSlope=parameters.defaultSlope
    ncfile.topoPosMinDiff=parameters.topoPosMinDiff
    ncfile.topoPosMaxDiff =parameters.topoPosMaxDiff
    ncfile.topoPosExp =parameters.topoPosExp
    ncfile.costalExp =parameters.coastalExp
    ncfile.layerExp =parameters.layerExp
    ncfile.distanceWeightScale =parameters.distanceWeightScale
    ncfile.distanceWeightExp =parameters.distanceWeightExp
    ncfile.maxGrad =parameters.maxGrad
    ncfile.bufferSlope =parameters.bufferSlope
    ncfile.minElev =parameters.minElev
    ncfile.minElevDiff =parameters.minElevDiff
    ncfile.recomputeDefaultPrecipSlope = parameters.recomputeDefaultPrecipSlope
    ncfile.filterSize =parameters.filterSize
    ncfile.filterSpread =parameters.filterSpread
    ncfile.covWindow =parameters.covWindow

In [49]:
ncfile.close()  # just to be safe, make sure dataset is not already open.

#output
saveOutput(controlVars.outputName,controlVars.variableEstimated,grid,metGrid,parameters)

remove C:\Users\askarzam\Downloads\TIER-master\data\output\caliOutput_prcp_13sta_test.nc
